In [2]:
import pandas as pd
import xlwt 
import xlsxwriter #导入读写EXCEL的库
import os
import sys
import re
import time
import datetime
import openpyxl
import xlrd
from openpyxl import load_workbook
from openpyxl.styles import Alignment
import xlrd
from openpyxl.utils import get_column_letter
import numpy as np
from openpyxl.styles import PatternFill, Border, Side, Font, Alignment

#设置pandas输出对齐
pd.set_option("display.unicode.ambiguous_as_wide",True)
pd.set_option("display.unicode.east_asian_width",True)

#设置单元格框线格式
thin = Side(border_style="thin", color="000000")
border = Border(top=thin, left=thin, right=thin, bottom=thin)


#输入文件夹路径，例：C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\DCR
#file_path = input("请粘贴文件夹路径:")
#TM_位数 = input("请输入条码位数:")
TM_位数 = 24
file_path = r"C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)"
test_dir_list = file_path.split("\\")#如自动输入，使用\进行字符串切片



#创建报告文件函数插入点

creation_time1 = datetime.datetime.now()
creation_time2 = creation_time1.strftime('%Y-%m-%d')
test_file_name_1 = str("%s测试报告_by_pxg_%s.xlsx" % (test_dir_list[-1], creation_time2))

#print(test_file_name)
#print(test_file_name_1)
#print("断点2")

#创建表格


#test_repoter = xlsxwriter.Workbook() 
test_repoter = xlsxwriter.Workbook("%s" % test_file_name_1) 
#创建sheet
worksheet_封面 = test_repoter.add_worksheet('封面')
worksheet_报告汇总 = test_repoter.add_worksheet('报告汇总')
worksheet_不同温度放电 = test_repoter.add_worksheet('不同温度放电')
worksheet_倍率放电 = test_repoter.add_worksheet('倍率放电')
worksheet_DCR = test_repoter.add_worksheet('DCR')
#headings = ["电芯条码", "初始容量", "1C_DCR(mΩ)", "3C_DCR(mΩ)", "5C_DCR(mΩ)", "8C_DCR(mΩ)", "10C_DCR(mΩ)", "判定"]
worksheet_能量效率 = test_repoter.add_worksheet('能量效率')
worksheet_存储 = test_repoter.add_worksheet('存储')
#u'\u2103'是℃符号
worksheet_45度工况循环 = test_repoter.add_worksheet('45℃工况循环')
worksheet_25度工况循环 = test_repoter.add_worksheet('25℃工况循环')
worksheet_25度倍率循环 = test_repoter.add_worksheet('25℃倍率循环')
worksheet_循环中DCR = test_repoter.add_worksheet('循环中DCR')

#设置单元格格式
property_加粗字体 = {
      'font_size': 11,#字体大小
      'bold':True, #是否加粗
      'align': 'center',#水平对齐方式
      'valign': 'center',#垂直对齐方式
      'font_name': u'微软雅黑',
      'text_wrap': False, # 是否自动换行
      'color': "black",#黑色
      'fg_color': '#F4B084',
      'border':1, #设置边框
      }
property_1 = {
      'font_size': 11,#字体大小
      'bold':False, #是否加粗
      'align': 'center',#水平对齐方式
      'valign': 'center',#垂直对齐方式
      'font_name': u'微软雅黑',
      'text_wrap': False, # 是否自动换行
      'color': "black",#蓝色
      #'fg_color': '#F4B084',
      'border':1, #设置边框
      }
#合并单元格
merge_format = test_repoter.add_format({
    'bold':     True,
    'border':   6,
    'align':    'center',#水平居中
    'valign':   'vcenter',#垂直居中
    'fg_color': '#D7E4BC',#颜色填充
})
#设置表头单元格格式
cell_format1 = test_repoter.add_format(property_加粗字体)
cell_format2 = test_repoter.add_format(property_1)
#设置DCR表头单元格格式
DCR_headings = ["电芯条码", "初始容量（Ah）", "1C_DCR(mΩ)", "3C_DCR(mΩ)", "5C_DCR(mΩ)", "8C_DCR(mΩ)", "10C_DCR(mΩ)", "判定"]
DCR_headings2 = ["/", "标准", 3.6, 2.7, 2.2, 2.0, 1.8, "/"]
worksheet_DCR.write_row("A1", DCR_headings, cell_format1)
worksheet_DCR.write_row("A2", DCR_headings2, cell_format2)
worksheet_DCR.set_column("A:H", 13)
worksheet_DCR.set_row(0, 30)
worksheet_DCR.write_row("A1", DCR_headings, cell_format1)
worksheet_DCR.write_row("A2", DCR_headings2, cell_format2)
#worksheet_DCR.set_column("A:H", 17)
#设置能量效率表头单元格格式
NLXL_headings = ["电芯条码", "温度（℃）", "SOC", "初始容量（Ah）", "放电前电压(V)", "放电后电压(V)", "放电能量(Wh)", "充电前电压(V)", "充电后电压(V)", "充电能量(Wh)", "能量效率","标准","判定"]
worksheet_能量效率.write_row("A1", DCR_headings, cell_format1)
#worksheet_能量效率.write_row("A2", DCR_headings2, cell_format2)
worksheet_能量效率.set_column("A:M", 13)
worksheet_能量效率.set_row(0, 30)
worksheet_能量效率.write_row("A1", NLXL_headings, cell_format1)
#worksheet_能量效率.write_row("A2", DCR_headings2, cell_format2)
#worksheet_能量效率.set_column("A:M", 17)
#设置不同温度表头单元格格式
worksheet_不同温度放电.merge_range('A1:I1', '不同温度容量', merge_format)
worksheet_不同温度放电.merge_range('A13:I13', '不同温度放电比率', merge_format)
worksheet_不同温度放电.merge_range('A26:I26', '不同温度放电温升', merge_format)

BTWD_headings1 = ["序号", "电芯条码", "55℃容量(Ah)", "25℃容量(Ah)", "0℃容量(Ah)", "-10℃容量(Ah)", "-20℃容量(Ah)", "-30℃容量(Ah)", "-40℃容量(Ah)"]
BTWD_headings2 = ["序号", "电芯条码", "55℃容量比率", "25℃容量比率", "0℃容量比率", "-10℃容量比率", "-20℃容量比率", "-30℃容量比率", "-40℃容量比率"]
BTWD_headings3 = ["标准", "/", 0.98, 0.98,0.98, 0.98, 0.98, 0.98, 0.98]
BTWD_headings4 = ["序号", "电芯条码", "55℃放电温升", "25℃放电温升", "0℃放电温升", "-10℃放电温升", "-20℃放电温升", "-30℃放电温升", "-40℃放电温升"]

worksheet_不同温度放电.write_row("A2", BTWD_headings1, cell_format1)
worksheet_不同温度放电.write_row("A14", BTWD_headings2, cell_format1)
worksheet_不同温度放电.write_row("A15", BTWD_headings3, cell_format1)
worksheet_不同温度放电.write_row("A27", BTWD_headings4, cell_format1)
#worksheet_能量效率.write_row("A2", DCR_headings2, cell_format2)
worksheet_不同温度放电.set_column("B:I", 13)
worksheet_不同温度放电.set_column("A:A", 8)
worksheet_不同温度放电.set_row(0, 20)
worksheet_不同温度放电.set_row(12, 20)
worksheet_不同温度放电.set_row(25, 20)
#worksheet_不同温度放电.write_row("A1", NLXL_headings, cell_format1)

#设置不同倍率放电报告表格表头格式
worksheet_倍率放电.merge_range('A1:G1', '倍率放电容量', merge_format)
worksheet_倍率放电.merge_range('A13:G13', '倍率放电容量比率', merge_format)
worksheet_倍率放电.merge_range('A26:G26', '倍率放电容量温升', merge_format)

BLFD_headings1 = ["序号", "电芯条码", "初始容量(Ah)", "10C容量(Ah)", "20C容量(Ah)", "30C容量(Ah)", "40C容量(Ah)"]
BLFD_headings2 = ["序号", "电芯条码", "1C比率", "10C比率", "20C比率", "30C比率", "40C比率"]
BLFD_headings3 = ["标准", "/","/", 0.96, 0.95,0.94, 0.94]
BLFD_headings4 = ["序号", "电芯条码", "1C温升", "10C温升", "20C温升", "30C温升", "40C温升"]

worksheet_倍率放电.write_row("A2", BLFD_headings1, cell_format1)
worksheet_倍率放电.write_row("A14", BLFD_headings2, cell_format1)
worksheet_倍率放电.write_row("A15", BLFD_headings3, cell_format1)
worksheet_倍率放电.write_row("A27", BLFD_headings4, cell_format1)
#worksheet_能量效率.write_row("A2", DCR_headings2, cell_format2)
worksheet_倍率放电.set_column("B:G", 13)
worksheet_倍率放电.set_column("A:A", 8)
worksheet_倍率放电.set_row(0, 20)
worksheet_倍率放电.set_row(12, 20)
worksheet_倍率放电.set_row(25, 20)



test_repoter.close()

repoter_path = os.getcwd()
repoter_file_path = os.path.join(repoter_path, test_file_name_1)
print("1#" * 30)#分隔线
print(repoter_path)
#print(repoter_file_path)
print("1#" * 30)


dq_dir_len = len(file_path)
#print(dq_dir_len)

num = 0
#遍历输入的测试数据文件夹

#设置计数器
counter_BLFD = 1
counter_SWFD = 1
counter_DWFD_55_T = 1
counter_DWFD_0_T = 1
counter_DWFD_10_T = 1
counter_DWFD_20_T = 1
counter_DWFD_30_T = 1
counter_DWFD_40_T = 1
counter_DCR = 1
counter_能量效率 = 1
counter_存储 = 1
counter_45度工况循环 = 1
counter_25度工况循环 = 1
counter_倍率循环 = 1

for dirpath, dirnames, filenames in os.walk(file_path):
    #print(num)
    
    for file_name_1 in filenames:
        print("读取第 %s 个文件.........." % num)
        #time.sleep(0.5)
        
        file_path = os.path.join(dirpath, file_name_1)
        #print(file_name_1)
        print(file_path)
        #print(filenames)
        
        dq_dir_len1 = len(dirpath)
        file_name_dq = dirpath [dq_dir_len + 1 : dq_dir_len1]
        print(file_name_dq)
        
            #判断文件夹是否是DCR
        if file_name_dq == "DCR":
            print("准备处理DCR数据")
            #获取10S脉冲放电电压数据
            sheetName_V = "record"
            df_DCR_V = pd.read_excel(file_path, sheet_name = sheetName_V)
            df_DCR_V_1 = df_DCR_V.iloc[:, [2, 3, 4, 5]]#索引切片，获取数据
            #print(df_DCR_V_1)
            df_rest_V_1 = df_DCR_V_1[df_DCR_V_1['记录时间(h:min:s.ms)'] == '2:00:00.000']
            first_rest_V_1 = df_rest_V_1.iloc[0, 1]
            #print(first_rest_V_1)
            #print(type(first_rest_V_1))
            
            #print(df_rest_V_1)
            df_rest_V_2 = df_DCR_V_1[df_DCR_V_1['记录时间(h:min:s.ms)'] == '0:00:10.000']
            df_rest_V_2_1 = df_rest_V_2[df_rest_V_2['记录序号'] > first_rest_V_1]
            #print(df_rest_V_2_1)
            
            #获取脉冲放电前电压的工步序号
            DCR_V_BD_1C = df_rest_V_1.iloc[0, 1] - 1
            DCR_V_BD_3C = df_rest_V_1.iloc[1, 1] - 1
            DCR_V_BD_5C = df_rest_V_1.iloc[2, 1] - 1
            DCR_V_BD_8C = df_rest_V_1.iloc[3, 1] - 1
            DCR_V_BD_10C = df_rest_V_1.iloc[4, 1] - 1
            
            #print(DCR_V_BD_1C)
            
            
            #获取脉冲放电后电压的工步序号
            DCR_V_AD_1C = df_rest_V_2_1.iloc[0, 1]
            DCR_V_AD_3C = df_rest_V_2_1.iloc[1, 1]
            DCR_V_AD_5C = df_rest_V_2_1.iloc[2, 1]
            DCR_V_AD_8C = df_rest_V_2_1.iloc[3, 1]
            DCR_V_AD_10C = df_rest_V_2_1.iloc[4, 1]
            
            #1C脉冲放电电压
            df_DCR_V_1C = df_DCR_V_1.iloc[DCR_V_BD_1C : DCR_V_AD_1C, 3] 
            np_DCR_V_1C = np.array(df_DCR_V_1C)#np.ndarray()
            #print(DCR_V_1C_list)
            #3C脉冲放电电压
            df_DCR_V_3C = df_DCR_V_1.iloc[DCR_V_BD_3C : DCR_V_AD_3C, 3] 
            np_DCR_V_3C = np.array(df_DCR_V_3C)#np.ndarray()
            #print(DCR_V_3C_list)
            #5C脉冲放电电压
            df_DCR_V_5C = df_DCR_V_1.iloc[DCR_V_BD_5C : DCR_V_AD_5C, 3] 
            np_DCR_V_5C = np.array(df_DCR_V_5C)#np.ndarray()
            #8C脉冲放电电压
            df_DCR_V_8C = df_DCR_V_1.iloc[DCR_V_BD_8C : DCR_V_AD_8C, 3] 
            np_DCR_V_8C = np.array(df_DCR_V_8C)#np.ndarray()
            #10C脉冲放电电压
            df_DCR_V_10C = df_DCR_V_1.iloc[DCR_V_BD_10C : DCR_V_AD_10C, 3] 
            np_DCR_V_10C = np.array(df_DCR_V_10C)#np.ndarray()
            #将获取的1C-10C电压数据转换为列表，方便后期写入excel
            DCR_V_1C_list = np_DCR_V_1C.tolist()#list
            DCR_V_3C_list = np_DCR_V_3C.tolist()#list
            DCR_V_5C_list = np_DCR_V_5C.tolist()#list
            DCR_V_8C_list = np_DCR_V_8C.tolist()#list
            DCR_V_10C_list = np_DCR_V_10C.tolist()#list
            #print(DCR_V_10C_list)
            
            DCR_V_10C_list_len = len(DCR_V_10C_list)
            
            #读取cycle表中的初始容量数据
            sheetName = "cycle"
            df_DCR = pd.read_excel(file_path, sheet_name = sheetName)
            #print(df_DCR)
            Co_DCR_df = df_DCR.iloc[1:4, 2:3]#索引切片
            #print(Co_DCR_df)#读取初始容量
            C_1 = Co_DCR_df.at[1, "放电容量(Ah)"]
            #print(C_1)
            C_2 = Co_DCR_df.at[2, "放电容量(Ah)"]
            #print(C_2)
            C_3 = Co_DCR_df.at[3, "放电容量(Ah)"]
            #print(C_3)
            Co_DCR = round((C_1 + C_2 + C_3) / 3, 4)           #取DCR数据的初始容量
            #print(Co_DCR)

           
            #读取DCR测试步骤电压数据
            sheetName_step = "step"
            df_DCR_Voltage = pd.read_excel(file_path, sheet_name = sheetName_step)
            
            #print(df_DCR_Voltage)
            #print(df_DCR_Voltage.index())
            
            #V_DCR_df = df_DCR_Voltage.iloc[ : , 2 : 8]#查找符合要求的索引
            V_DCR_df = df_DCR_Voltage.iloc[ : , [2, 3, 6, 7]]#查找符合要求的索引
            #print(V_DCR_df)#读取电压数据
            
            #取静置最后一秒电压
            V_DCR_df_rest = V_DCR_df[V_DCR_df['工步时间(h:min:s.ms)'] == '2:00:00.000']
            #print(V_DCR_df_rest)
            V_DCR_df_1C_rest = V_DCR_df_rest.iloc[0, 3]
            V_DCR_df_3C_rest = V_DCR_df_rest.iloc[1, 3]
            V_DCR_df_5C_rest = V_DCR_df_rest.iloc[2, 3]
            V_DCR_df_8C_rest = V_DCR_df_rest.iloc[3, 3]
            V_DCR_df_10C_rest = V_DCR_df_rest.iloc[4, 3]
  
            #取放电最后一秒的数据
            V_DCR_df_DC = V_DCR_df[V_DCR_df['工步时间(h:min:s.ms)'] == '0:00:10.000']
            #print(V_DCR_df_DC)
            #print(type(V_DCR_df_DC))
            V_DCR_df_1C_DC = V_DCR_df_DC.iloc[0, 3]
            V_DCR_df_3C_DC = V_DCR_df_DC.iloc[1, 3]
            V_DCR_df_5C_DC = V_DCR_df_DC.iloc[2, 3]
            V_DCR_df_8C_DC = V_DCR_df_DC.iloc[3, 3]
            V_DCR_df_10C_DC = V_DCR_df_DC.iloc[4, 3]

            #计算DCR，并保留2位小数
            
            DCR_1C = round((V_DCR_df_1C_rest - V_DCR_df_1C_DC) / Co_DCR * 1000, 2)
            DCR_3C = round((V_DCR_df_3C_rest - V_DCR_df_3C_DC) / Co_DCR * 1000 / 3, 2)
            DCR_5C = round((V_DCR_df_5C_rest - V_DCR_df_5C_DC) / Co_DCR * 1000 / 5, 2)
            DCR_8C = round((V_DCR_df_8C_rest - V_DCR_df_8C_DC) / Co_DCR * 1000 / 8, 2)
            DCR_10C = round((V_DCR_df_3C_rest - V_DCR_df_10C_DC) / Co_DCR * 1000 / 10, 2)
            #print("DCR: ")
            #print(DCR_1C, DCR_3C, DCR_5C, DCR_8C, DCR_10C)
            
            cell_num = file_name_1[:-4]
            #print(cell_num)
 
            #DCR_headings = ["电芯条码", "初始容量", "1C_DCR(mΩ)", "3C_DCR(mΩ)", "5C_DCR(mΩ)", "8C_DCR(mΩ)", "10C_DCR(mΩ)"]
            
            #建立DCR表格DCR数据列表
            DCR_data_list = []
            
            DCR_data_list.append(str("%s" % cell_num))
            DCR_data_list.append(float("%s" % Co_DCR))
            DCR_data_list.append(float("%s" % DCR_1C))
            DCR_data_list.append(float("%s" % DCR_3C))
            DCR_data_list.append(float("%s" % DCR_5C))
            DCR_data_list.append(float("%s" % DCR_8C))
            DCR_data_list.append(float("%s" % DCR_10C))
    
            wb_DCR = openpyxl.load_workbook("%s" % test_file_name_1)
            ws_DCR = wb_DCR["DCR"]
            #print(ws_DCR)
            #DCR_data = np.array(DCR_data_list)
            
            #填入DCR数据
            for i in range(1, 8):
                #for col in range("%s" % row_DCR, "%s" % row_DCR)
                k_DCR = i - 1
                ws_DCR.cell(row = counter_DCR +2,column = i, value = DCR_data_list[k_DCR])
                ws_DCR.cell(row = 12,column = counter_DCR, value = "%s-10C电压(V)" % cell_num)
                ws_DCR.cell(row = 12,column = i).alignment = Alignment(horizontal='right', vertical='center')
                thin_DCR = Side(border_style="thin", color="000000")
                border_DCR = Border(top=thin_DCR, left=thin_DCR, right=thin_DCR, bottom=thin_DCR)
                ws_DCR.cell(row = counter_DCR +2,column = i).alignment = Alignment(horizontal='center', vertical='center')
                ws_DCR.cell(row = counter_DCR +2,column = i+1).alignment = Alignment(horizontal='center', vertical='center')
                ws_DCR.cell(row=counter_DCR +2, column=i).border = border_DCR
                ws_DCR.cell(row=counter_DCR +2, column= i+1).border = border_DCR
            #判断DCR是否合格
            if ws_DCR.cell(row = counter_DCR +2,column = 3).value < ws_DCR["C2"].value and ws_DCR.cell(row = counter_DCR +2,column = 4).value < ws_DCR["D2"].value and ws_DCR.cell(row = counter_DCR +2,column = 5).value < ws_DCR["E2"].value and ws_DCR.cell(row = counter_DCR +2,column = 6).value < ws_DCR["F2"].value and ws_DCR.cell(row = counter_DCR +2,column = 7).value < ws_DCR["G2"].value:
                ws_DCR.cell(row = counter_DCR +2,column = 8, value = "合格")
            else:
                ws_DCR.cell(row = counter_DCR +2,column = 8, value = "不合格")
                
            #填入电压数据    
            for j in range(1, DCR_V_10C_list_len+1):
                ws_DCR.cell(row = j+12, column = counter_DCR, value = DCR_V_10C_list[j-1])
            ws_DCR.cell(row = counter_DCR +2,column = 1).alignment = Alignment(horizontal='right', vertical='center')
            
            #计数器+1
            counter_DCR += 1

            #print(ws_DCR)
            wb_DCR.save("%s" % test_file_name_1)
            #print("1#" * 30)
            print("1#" * 30)
            
            
           
 
        #判断文件夹是否是不同温度放电\25℃
        elif file_name_dq == "不同温度放电\\25℃":
            print("准备处理不同温度放电25℃数据")
            
            SWFD_cell_num = file_name_1[ : TM_位数]
            print(SWFD_cell_num)
            
            #输入工作簿名字，准备赋值给DataFrame
            #获取循环数据工作簿名字
            sheetName_SW_C = "cycle"
            #获取详细数据工作簿名字
            sheetName_SW_V = "record"
            #获取温度数据工作簿名字
            sheetName_SW_T = "auxTemperature"
            
            #用DataFrame取循环数据
            df_SWFD_1 = pd.read_excel(file_path, sheet_name = sheetName_SW_C)
            #print(df_SWFD_1)
            df_SWFD_2 = df_SWFD_1.iloc[:, [2]]#索引切片（第3列数据），获取放电容量数据
            #print(df_SWFD_2)
            df_SWFD_3 = df_SWFD_2[df_SWFD_2["放电容量(Ah)"] > 8]#索引切片（容量大于8的数据），获取初始3次容量数据
            #print(df_SWFD_3)
            #np_SWFD_C_25度 = np.array(df_SWFD_3)#将二维DataFrame数组通过np.ndarray()转换成一维数据
            #将获取的容量数据转换为列表，方便计算初始容量和后期写入excel
            #SWFD_C_25度_list = np_SWFD_C_25度.tolist()#list
            #print(SWFD_C_25度_list)
            #三次容量求和
            SWFD_C1 = df_SWFD_3.at[1, "放电容量(Ah)"]
            SWFD_C2 = df_SWFD_3.at[2, "放电容量(Ah)"]
            SWFD_C3 = df_SWFD_3.at[3, "放电容量(Ah)"]
            #print(SWFD_C1, SWFD_C2, SWFD_C3)
            #计算不同温度放电初始容量
            SWFD_C = round((SWFD_C1 + SWFD_C2+ SWFD_C3)/3, 4)
            #print(SWFD_C)
            
            #用DataFrame取温度数据
            df_SWFD_T_1 = pd.read_excel(file_path, sheet_name = sheetName_SW_T)
            df_SWFD_T_2 = df_SWFD_T_1.iloc[:, [0, 2]]
            #print(df_SWFD_T_2)
            #用DataFrame取电压数据
            df_SWFD_V_1 = pd.read_excel(file_path, sheet_name = sheetName_SW_V)
            df_SWFD_V_2 = df_SWFD_V_1.iloc[:, 1:8]
            #print(df_SWFD_V_2)
            
            #将两张表数据汇总到一起进行电压和温度数据的提取
            df_SWFD_1 = pd.merge(df_SWFD_V_2, df_SWFD_T_2, on='记录序号')
            #print(df_SWFD_1)
            
            #通过工步数据筛选所需数据起点
            df_SWFD_2 = df_SWFD_1[df_SWFD_1["电压(V)"] <= 2]
            #print(df_SWFD_2)
            #获取最后一个放电末端工步序号，便于后面切片取所需要的数据
            SWFD_No_AD_1 = df_SWFD_2.iloc[3, 0]
            #print(SWFD_No_AD_1)
            #获取最后一个放电末端行号
            #数据终点行号
            SWFD_No_AD_2 = df_SWFD_2.iloc[3, 2]
            #print(SWFD_No_AD_2)
            #获取最后一个放电容量数据中第一个记录点的行号，便于后面切片取所需要的数据
            df_SWFD_No_AD_3 = df_SWFD_1[df_SWFD_1["工步序号"] == SWFD_No_AD_1]
            #数据起点行号
            SWFD_No_AD_3 = df_SWFD_No_AD_3.iloc[0, 2] - 2
            #切片取所需列数据
            df_SWFD_No_AD_4 = df_SWFD_1.iloc[:, [4, 6, 7]]
            #切片取所需行数据,准备填入工作表
            df_SWFD = df_SWFD_No_AD_4.iloc[SWFD_No_AD_3 : SWFD_No_AD_2, :]
            #print(df_SWFD)
            
            #取电压数据一列
            df_SWFD_V_1 = df_SWFD.iloc[ : , 0] 
            np_SWFD_V_1 = np.array(df_SWFD_V_1)#np.ndarray()
            #将获取的电压数据转换为列表，方便后期写入excel
            SWFD_V_1_list = np_SWFD_V_1.tolist()#list
            
            #取容量数据一列
            df_SWFD_C_1 = df_SWFD.iloc[ : , 1] 
            np_SWFD_C_1 = np.array(df_SWFD_C_1)#np.ndarray()
            #将获取的容量数据转换为列表，方便后期写入excel
            SWFD_C_1_list = np_SWFD_C_1.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            SWFD_C_1 = SWFD_C_1_list[-1]
            #数据列表长度
            SWFD_C_1_list_len = len(SWFD_C_1_list)
            #print(SWFD_C_1_list_len)
            
            #取温度数据一列
            df_SWFD_T_1 = df_SWFD.iloc[ : , 2] 
            np_SWFD_T_1 = np.array(df_SWFD_T_1)#np.ndarray()
            #将获取的温度数据转换为列表，方便后期写入excel
            SWFD_T_1_list = np_SWFD_T_1.tolist()#list
            #取第一个和最后一个温度值作为计算当前温度下放电温升的数据
            SWFD_T_1 = SWFD_T_1_list[0]
            SWFD_T_2 = SWFD_T_1_list[-1]
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            SWFD_T_2_list = []
            for i_SWFD_T in range(0,SWFD_C_1_list_len):
                SWFD_T_2_list_i = SWFD_T_1_list[i_SWFD_T] - SWFD_T_1
                SWFD_T_2_list.append(SWFD_T_2_list_i)
            #print(SWFD_T_2_list)
            #计算当前温度放电温升
            SWFD_T_rise = round(SWFD_T_2 - SWFD_T_1, 2)
            
            
            
            #打印输出所取数据，准备填入excel表
            #print(SWFD_C_1)
            #print(SWFD_T_rise)
            #print(SWFD_C_1_list)
            #print(SWFD_T_1_list)
            #print(SWFD_V_1_list)
            
            
            #准备打开报告，写入25℃放电容量数据
            wb_SWFD = openpyxl.load_workbook("%s" % test_file_name_1)
            ws_SWFD = wb_SWFD["不同温度放电"]
            #填入序号、条码、容量、温升数据
            #k_SWFD_3 = counter_SWFD +27
            #k_SWFD_4 = counter_SWFD + 9 + (counter_SWFD - 1) * 28
            #k_SWFD_5 = counter_SWFD + 9 + (counter_SWFD - 1) * 28 + 28
            #指定写入绘图表头的初始列索引
            #k_SWFD_6 = get_column_letter(k_SWFD_4)
            #k_SWFD_7 = get_column_letter(k_SWFD_5)
            #指定写入25℃数据的初始列索引
            k_SWFD_1 = counter_SWFD + 12 + (counter_SWFD - 1) * 27
            #设置放电容量表格格式
            for i_SWFD in ws_SWFD["A%s:I%s" % (counter_SWFD +2, counter_SWFD +2)]:
                for cell in i_SWFD:
                    cell.border = border#区域单元格设置边框
                    cell.alignment = Alignment(horizontal='center', vertical='center')
            #设置放电比率表格格式        
            for i_SWFD_2 in ws_SWFD["A%s:I%s" % (counter_SWFD +15, counter_SWFD +15)]:
                for cell_2 in i_SWFD_2:
                    cell_2.border = border#区域单元格设置边框
                    cell_2.alignment = Alignment(horizontal='center', vertical='center')
            #设置放电温升表格格式        
            for i_SWFD_3 in ws_SWFD["A%s:I%s" % (counter_SWFD +27, counter_SWFD +27)]:
                for cell_3 in i_SWFD_3:
                    cell_3.border = border#区域单元格设置边框
                    cell_3.alignment = Alignment(horizontal='center', vertical='center')
            
            #写入数据
            ws_SWFD.cell(row = 1,column = k_SWFD_1 - 3, value = SWFD_cell_num)
            for i_SWFD_5 in range(1, 5):
                #for col in range("%s" % row_DCR, "%s" % row_DCR)
                SWFD_col_list = ["25℃放电容量（Ah）", "25℃放电电压（V）", "25℃放电温度（℃）", "25℃放电温升（℃）"]
                ws_SWFD.cell(row = 2,column = k_SWFD_1+i_SWFD_5, value = SWFD_col_list[i_SWFD_5 -1])
                
            for j_SWFD in range(1, SWFD_C_1_list_len+1):
                #写入容量、电压、温度、温升四列数据
                ws_SWFD.cell(row = j_SWFD+2, column = k_SWFD_1+1, value = SWFD_C_1_list[j_SWFD-1]) 
                ws_SWFD.cell(row = j_SWFD+2, column = k_SWFD_1+2, value = SWFD_V_1_list[j_SWFD-1])
                ws_SWFD.cell(row = j_SWFD+2, column = k_SWFD_1+3, value = SWFD_T_1_list[j_SWFD-1])
                ws_SWFD.cell(row = j_SWFD+2, column = k_SWFD_1+4, value = SWFD_T_2_list[j_SWFD-1])
            #写入容量表    
            ws_SWFD.cell(row = counter_SWFD +2,column = 1, value = counter_SWFD)
            ws_SWFD.cell(row = counter_SWFD +2,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_SWFD.cell(row = counter_SWFD +2,column = 2, value = SWFD_cell_num)
            ws_SWFD.cell(row = counter_SWFD +2,column = 4, value = SWFD_C)
            #写入比率表    
            ws_SWFD.cell(row = counter_SWFD +15,column = 1, value = counter_SWFD)
            ws_SWFD.cell(row = counter_SWFD +15,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_SWFD.cell(row = counter_SWFD +15,column = 2, value = SWFD_cell_num)
            ws_SWFD.cell(row = counter_SWFD +15,column = 4, value = 1)
            #写入温升表    
            ws_SWFD.cell(row = counter_SWFD +27,column = 1, value = counter_SWFD)
            ws_SWFD.cell(row = counter_SWFD +27,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_SWFD.cell(row = counter_SWFD +27,column = 2, value = SWFD_cell_num)
            ws_SWFD.cell(row = counter_SWFD +27,column = 4, value = SWFD_T_rise)
            
           
                
            #填入电压数据    
            #for j in range(1, DCR_V_10C_list_len+1):
                #ws_DCR.cell(row = j+12, column = counter_DCR, value = DCR_V_10C_list[j-1])
            #ws_DCR.cell(row = counter_DCR +2,column = 1).alignment = Alignment(horizontal='right', vertical='center')
            
            #计数器+1
            counter_SWFD += 1

            #print(ws_SWFD)
            wb_SWFD.save("%s" % test_file_name_1)
            print("1#" * 30)
            print("1#" * 30)
           
            
            
            
            
            #SWFD_C1 = SWFD_C_25度_list
            
            
            
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        #判断不同温度放电\0℃
        elif file_name_dq == "不同温度放电\\0℃":
            print("准备调用不同温度放电0℃.py")
            #判断不同温度放电\0℃
            DWFD_0_T_cell_num = file_name_1[ : TM_位数]
            print(DWFD_0_T_cell_num)
            #打开数据
            df_DWFD_0_1 = pd.ExcelFile(file_path)
            DWFD_0_sheetnames_list = df_DWFD_0_1.sheet_names  # 获取工作表名称
            DWFD_0_sheetname = DWFD_0_sheetnames_list[1]
            #读取详细数据文件
            df_DWFD_0_2 = pd.read_excel(file_path, sheet_name = DWFD_0_sheetname)
            df_DWFD_0_3 = df_DWFD_0_2.iloc[:, [5, 6, 8, 13]]#索引切片，获取需要的电压、放电容量、温升数据
            df_DWFD_0_4 = df_DWFD_0_3[df_DWFD_0_3["Current(A)"] <= -9]
            
            #获取开头静置数据行数
            df_DWFD_0_5 = df_DWFD_0_3[df_DWFD_0_3["Discharge_Capacity(Ah)"] <= 0]
            #print(df_DWFD_0_5)
            #获取容量一列数据
            df_DWFD_0_C_1 = df_DWFD_0_5.iloc[ : , 2]
            np_DWFD_0_C_1 = np.array(df_DWFD_0_C_1)#np.ndarray()
            DWFD_0_C_1_list = np_DWFD_0_C_1.tolist()
            #取静置最后一秒数据索引号
            num_DWFD_0_df_1 = len(DWFD_0_C_1_list)-1
            #取电压数据一列
            df_DWFD_0_V_1 = df_DWFD_0_4.iloc[ : , 0] 
            #获取低温放电数据行数
            np_DWFD_0_V_1 = np.array(df_DWFD_0_V_1)#np.ndarray()
            DWFD_0_V_1_list = np_DWFD_0_V_1.tolist()
            num_DWFD_0_df = len(DWFD_0_V_1_list)+1
            #print(num_DWFD_0_df_1)
            #print(num_DWFD_0_df)
            
            #索引切片截取所需数据段(行、列)
            df_DWFD_0_6 = df_DWFD_0_3.iloc[num_DWFD_0_df_1:num_DWFD_0_df+num_DWFD_0_df_1,:]
            #print(df_DWFD_0_6)
            
            #获取待写入报告的容量数据
            df_DWFD_0_C = df_DWFD_0_6.iloc[:,2]
            np_DWFD_0_C = np.array(df_DWFD_0_C)#np.ndarray()
            #print(df_DWFD_0_C)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_0_T_C_list = np_DWFD_0_C.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            DWFD_0_T_C = DWFD_0_T_C_list[-1]
            #数据列表长度,后期向报告单元格写入这一列数据需要用到，作为遍历循环的上限值
            DWFD_0_T_C_list_len = len(DWFD_0_T_C_list)
            #print(DWFD_0_T_C_list_len)
            
            #获取待写入报告的电压数据
            df_DWFD_0_V = df_DWFD_0_6.iloc[:,0]
            np_DWFD_0_V = np.array(df_DWFD_0_V)#np.ndarray()
            #print(df_DWFD_0_V)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_0_T_V_list = np_DWFD_0_V.tolist()#list
            
            
            #获取待写入报告的温度数据
            df_DWFD_0_T = df_DWFD_0_6.iloc[:,3]
            np_DWFD_0_T = np.array(df_DWFD_0_T)#np.ndarray()
            #print(df_DWFD_0_T)
            #将获取的温度数据转换为列表，方便后期写入excel
            DWFD_0_T_T_list = np_DWFD_0_T.tolist()#list
            #取第一个温度值
            DWFD_0_T_T_1 = DWFD_0_T_T_list[0]
            #取最后一个温度值，计算放电温升
            DWFD_0_T_T_2 = DWFD_0_T_T_list[-1]
            DWFD_0_T_rise = DWFD_0_T_T_2 - DWFD_0_T_T_1
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            DWFD_0_T_T_2_list = []
            for i_DWFD_0_T_T in range(0,DWFD_0_T_C_list_len):
                #遍历温度列表，减去初始温度，得到温升值，温升值用round取两位小数
                DWFD_0_T_T_2_list_i = round(DWFD_0_T_T_list[i_DWFD_0_T_T] - DWFD_0_T_T_1, 2)
                DWFD_0_T_T_2_list.append(DWFD_0_T_T_2_list_i)
            #print(DWFD_0_T_T_2_list)
            
           
            #打开报告,写入数据
            #准备打开报告，写入0℃放电容量数据
            wb_DWFD_0_T = openpyxl.load_workbook("%s" % test_file_name_1)
            ws_DWFD_0_T = wb_DWFD_0_T["不同温度放电"]
            #填入序号、条码、容量、温升数据
            #k_SWFD_3 = counter_SWFD +27
            #k_DWFD_0_T_4 = counter_DWFD_0_T + 9 + (counter_DWFD_0_T - 1) * 28  #10lie
            #k_DWFD_0_T_5 = counter_DWFD_0_T + 9 + (counter_DWFD_0_T - 1) * 28 + 28  #38列
            #指定写入绘图表头的初始列索引
            #k_DWFD_0_T_6 = get_column_letter(k_SWFD_4)
            #k_DWFD_0_T_7 = get_column_letter(k_SWFD_5)
            #指定写入0℃数据的初始列索引
            
            k_DWFD_0_T_1 = counter_DWFD_0_T + 12 + (counter_DWFD_0_T - 1) * 27 + 4  #17列
            
            #写入数据
            #ws_DWFD_0_T.cell(row = 1,column = k_DWFD_0_T_4, value = DWFD_0_T_cell_num)
            
            #if ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).value == DWFD_0_T_cell_num:
                
            for i_DWFD_0_T_5 in range(1, 5):
                #for col in range("%s" % row_DCR, "%s" % row_DCR)
                DWFD_0_T_col_list = ["0℃容量（Ah）", "0℃电压（V）", "0℃温度（℃）", "0℃温升（℃）"]
                ws_DWFD_0_T.cell(row = 2,column = k_DWFD_0_T_1+i_DWFD_0_T_5, value = DWFD_0_T_col_list[i_DWFD_0_T_5 -1])

            for j_DWFD_0_T in range(1, DWFD_0_T_C_list_len+1):
                #写入容量、电压、温度、温升四列数据
                ws_DWFD_0_T.cell(row = j_DWFD_0_T+2, column = k_DWFD_0_T_1+1, value = DWFD_0_T_C_list[j_DWFD_0_T-1]) 
                ws_DWFD_0_T.cell(row = j_DWFD_0_T+2, column = k_DWFD_0_T_1+2, value = DWFD_0_T_V_list[j_DWFD_0_T-1])
                ws_DWFD_0_T.cell(row = j_DWFD_0_T+2, column = k_DWFD_0_T_1+3, value = DWFD_0_T_T_list[j_DWFD_0_T-1])
                ws_DWFD_0_T.cell(row = j_DWFD_0_T+2, column = k_DWFD_0_T_1+4, value = DWFD_0_T_T_2_list[j_DWFD_0_T-1])
        #写入容量表  

            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 5, value = DWFD_0_T_C)
            #写入比率表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            
            ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 5, value = "=E%s/D%s" % (counter_DWFD_0_T +2, counter_DWFD_0_T +2))
            #写入温升表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 5, value = DWFD_0_T_rise)
            #else:
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 5, value = "请检查是否缺数据或者条码错误")

           
                
            #填入电压数据    
            #for j in range(1, DCR_V_10C_list_len+1):
                #ws_DCR.cell(row = j+12, column = counter_DCR, value = DCR_V_10C_list[j-1])
            #ws_DCR.cell(row = counter_DCR +2,column = 1).alignment = Alignment(horizontal='right', vertical='center')
            
            #计数器+1
            counter_DWFD_0_T += 1

            #print(ws_SWFD)
            wb_DWFD_0_T.save("%s" % test_file_name_1)
            print("1#" * 30)
            print("1#" * 30)
           

        #判断不同温度放电\-10℃
        elif file_name_dq == "不同温度放电\\-10℃":
            print("准备调用不同温度放电-10℃.py")
            #判断不同温度放电\-10℃
            DWFD_10_T_cell_num = file_name_1[ : TM_位数]
            print(DWFD_10_T_cell_num)
            #打开数据
            df_DWFD_10_1 = pd.ExcelFile(file_path)
            DWFD_10_sheetnames_list = df_DWFD_10_1.sheet_names  # 获取工作表名称
            DWFD_10_sheetname = DWFD_10_sheetnames_list[1]
            #读取详细数据文件
            df_DWFD_10_2 = pd.read_excel(file_path, sheet_name = DWFD_10_sheetname)
            df_DWFD_10_3 = df_DWFD_10_2.iloc[:, [5, 6, 8, 13]]#索引切片，获取需要的电压、放电容量、温升数据
            df_DWFD_10_4 = df_DWFD_10_3[df_DWFD_10_3["Current(A)"] <= -9]
            
            #获取开头静置数据行数
            df_DWFD_10_5 = df_DWFD_10_3[df_DWFD_10_3["Discharge_Capacity(Ah)"] <= 0]
            #print(df_DWFD_0_5)
            #获取容量一列数据
            df_DWFD_10_C_1 = df_DWFD_10_5.iloc[ : , 2]
            np_DWFD_10_C_1 = np.array(df_DWFD_10_C_1)#np.ndarray()
            DWFD_10_C_1_list = np_DWFD_10_C_1.tolist()
            #取静置最后一秒数据索引号
            num_DWFD_10_df_1 = len(DWFD_10_C_1_list)-1
            #取电压数据一列
            df_DWFD_10_V_1 = df_DWFD_10_4.iloc[ : , 0] 
            #获取低温放电数据行数
            np_DWFD_10_V_1 = np.array(df_DWFD_10_V_1)#np.ndarray()
            DWFD_10_V_1_list = np_DWFD_10_V_1.tolist()
            num_DWFD_10_df = len(DWFD_10_V_1_list)+1
            #print(num_DWFD_0_df_1)
            #print(num_DWFD_0_df)
            
            #索引切片截取所需数据段(行、列)
            df_DWFD_10_6 = df_DWFD_10_3.iloc[num_DWFD_10_df_1:num_DWFD_10_df+num_DWFD_10_df_1,:]
            #print(df_DWFD_0_6)
            
            #获取待写入报告的容量数据
            df_DWFD_10_C = df_DWFD_10_6.iloc[:,2]
            np_DWFD_10_C = np.array(df_DWFD_10_C)#np.ndarray()
            #print(df_DWFD_0_C)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_10_T_C_list = np_DWFD_10_C.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            DWFD_10_T_C = DWFD_10_T_C_list[-1]
            #数据列表长度,后期向报告单元格写入这一列数据需要用到，作为遍历循环的上限值
            DWFD_10_T_C_list_len = len(DWFD_10_T_C_list)
            #print(DWFD_0_T_C_list_len)
            
            #获取待写入报告的电压数据
            df_DWFD_10_V = df_DWFD_10_6.iloc[:,0]
            np_DWFD_10_V = np.array(df_DWFD_10_V)#np.ndarray()
            #print(df_DWFD_0_V)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_10_T_V_list = np_DWFD_10_V.tolist()#list
            
            
            #获取待写入报告的温度数据
            df_DWFD_10_T = df_DWFD_10_6.iloc[:,3]
            np_DWFD_10_T = np.array(df_DWFD_10_T)#np.ndarray()
            #print(df_DWFD_0_T)
            #将获取的温度数据转换为列表，方便后期写入excel
            DWFD_10_T_T_list = np_DWFD_10_T.tolist()#list
            #取第一个温度值
            DWFD_10_T_T_1 = DWFD_10_T_T_list[0]
            #取最后一个温度值，计算放电温升
            DWFD_10_T_T_2 = DWFD_10_T_T_list[-1]
            DWFD_10_T_rise = DWFD_10_T_T_2 - DWFD_10_T_T_1
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            DWFD_10_T_T_2_list = []
            for i_DWFD_10_T_T in range(0,DWFD_10_T_C_list_len):
                #遍历温度列表，减去初始温度，得到温升值，温升值用round取两位小数
                DWFD_10_T_T_2_list_i = round(DWFD_10_T_T_list[i_DWFD_10_T_T] - DWFD_10_T_T_1, 2)
                DWFD_10_T_T_2_list.append(DWFD_10_T_T_2_list_i)
            #print(DWFD_0_T_T_2_list)
            
           
            #打开报告,写入数据
            #准备打开报告，写入-10℃放电容量数据
            wb_DWFD_10_T = openpyxl.load_workbook("%s" % test_file_name_1)
            ws_DWFD_10_T = wb_DWFD_10_T["不同温度放电"]
            #填入序号、条码、容量、温升数据
            #k_SWFD_3 = counter_SWFD +27
            #k_DWFD_10_T_4 = counter_DWFD_10_T + 9 + (counter_DWFD_10_T - 1) * 28  #10lie
            #k_DWFD_10_T_5 = counter_DWFD_10_T + 9 + (counter_DWFD_10_T - 1) * 28 + 28  #38列
            #指定写入绘图表头的初始列索引
            #k_DWFD_0_T_6 = get_column_letter(k_SWFD_4)
            #k_DWFD_0_T_7 = get_column_letter(k_SWFD_5)
            #指定写入-10℃数据的初始列索引
            
            k_DWFD_10_T_1 = counter_DWFD_10_T + 12 + (counter_DWFD_10_T - 1) * 27 + 8  #21列
            
            #写入数据
            #ws_DWFD_0_T.cell(row = 1,column = k_DWFD_0_T_4, value = DWFD_0_T_cell_num)
            
            #if ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).value == DWFD_0_T_cell_num:
                
            for i_DWFD_10_T_5 in range(1, 5):
                #for col in range("%s" % row_DCR, "%s" % row_DCR)
                DWFD_10_T_col_list = ["-10℃容量（Ah）", "-10℃电压（V）", "-10℃温度（℃）", "-10℃温升（℃）"]
                ws_DWFD_10_T.cell(row = 2,column = k_DWFD_10_T_1+i_DWFD_10_T_5, value = DWFD_10_T_col_list[i_DWFD_10_T_5 -1])

            for j_DWFD_10_T in range(1, DWFD_10_T_C_list_len+1):
                #写入容量、电压、温度、温升四列数据
                ws_DWFD_10_T.cell(row = j_DWFD_10_T+2, column = k_DWFD_10_T_1+1, value = DWFD_10_T_C_list[j_DWFD_10_T-1]) 
                ws_DWFD_10_T.cell(row = j_DWFD_10_T+2, column = k_DWFD_10_T_1+2, value = DWFD_10_T_V_list[j_DWFD_10_T-1])
                ws_DWFD_10_T.cell(row = j_DWFD_10_T+2, column = k_DWFD_10_T_1+3, value = DWFD_10_T_T_list[j_DWFD_10_T-1])
                ws_DWFD_10_T.cell(row = j_DWFD_10_T+2, column = k_DWFD_10_T_1+4, value = DWFD_10_T_T_2_list[j_DWFD_10_T-1])
                
            #写入容量表  
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_10_T.cell(row = counter_DWFD_10_T +2,column = 6, value = DWFD_10_T_C)
            #写入比率表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            
            ws_DWFD_10_T.cell(row = counter_DWFD_10_T +15,column = 6, value = "=F%s/D%s" % (counter_DWFD_10_T +2, counter_DWFD_10_T +2))
            #写入温升表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_10_T.cell(row = counter_DWFD_10_T +27,column = 6, value = DWFD_10_T_rise)
            #else:
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 5, value = "请检查是否缺数据或者条码错误")

           
                
            #填入电压数据    
            #for j in range(1, DCR_V_10C_list_len+1):
                #ws_DCR.cell(row = j+12, column = counter_DCR, value = DCR_V_10C_list[j-1])
            #ws_DCR.cell(row = counter_DCR +2,column = 1).alignment = Alignment(horizontal='right', vertical='center')
            
            #计数器+1
            counter_DWFD_10_T += 1

            #print(ws_SWFD)
            wb_DWFD_10_T.save("%s" % test_file_name_1)
            print("1#" * 30)
            print("1#" * 30)
            
        #判断不同温度放电\-20℃
        elif file_name_dq == "不同温度放电\\-20℃":
            print("准备调用不同温度放电-20℃.py")
            #判断不同温度放电\-20℃
            DWFD_20_T_cell_num = file_name_1[ : TM_位数]
            print(DWFD_20_T_cell_num)
            #打开数据
            df_DWFD_20_1 = pd.ExcelFile(file_path)
            DWFD_20_sheetnames_list = df_DWFD_20_1.sheet_names  # 获取工作表名称
            DWFD_20_sheetname = DWFD_20_sheetnames_list[1]
            #读取详细数据文件
            df_DWFD_20_2 = pd.read_excel(file_path, sheet_name = DWFD_20_sheetname)
            df_DWFD_20_3 = df_DWFD_20_2.iloc[:, [5, 6, 8, 13]]#索引切片，获取需要的电压、放电容量、温升数据
            df_DWFD_20_4 = df_DWFD_20_3[df_DWFD_20_3["Current(A)"] <= -9]
            
            #获取开头静置数据行数
            df_DWFD_20_5 = df_DWFD_20_3[df_DWFD_20_3["Discharge_Capacity(Ah)"] <= 0]
            #print(df_DWFD_0_5)
            #获取容量一列数据
            df_DWFD_20_C_1 = df_DWFD_20_5.iloc[ : , 2]
            np_DWFD_20_C_1 = np.array(df_DWFD_20_C_1)#np.ndarray()
            DWFD_20_C_1_list = np_DWFD_20_C_1.tolist()
            #取静置最后一秒数据索引号
            num_DWFD_20_df_1 = len(DWFD_20_C_1_list)-1
            #取电压数据一列
            df_DWFD_20_V_1 = df_DWFD_20_4.iloc[ : , 0] 
            #获取低温放电数据行数
            np_DWFD_20_V_1 = np.array(df_DWFD_20_V_1)#np.ndarray()
            DWFD_20_V_1_list = np_DWFD_20_V_1.tolist()
            num_DWFD_20_df = len(DWFD_20_V_1_list)+1
            #print(num_DWFD_0_df_1)
            #print(num_DWFD_0_df)
            
            #索引切片截取所需数据段(行、列)
            df_DWFD_20_6 = df_DWFD_20_3.iloc[num_DWFD_20_df_1:num_DWFD_20_df+num_DWFD_20_df_1,:]
            #print(df_DWFD_0_6)
            
            #获取待写入报告的容量数据
            df_DWFD_20_C = df_DWFD_20_6.iloc[:,2]
            np_DWFD_20_C = np.array(df_DWFD_20_C)#np.ndarray()
            #print(df_DWFD_0_C)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_20_T_C_list = np_DWFD_20_C.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            DWFD_20_T_C = DWFD_20_T_C_list[-1]
            #数据列表长度,后期向报告单元格写入这一列数据需要用到，作为遍历循环的上限值
            DWFD_20_T_C_list_len = len(DWFD_20_T_C_list)
            #print(DWFD_0_T_C_list_len)
            
            #获取待写入报告的电压数据
            df_DWFD_20_V = df_DWFD_20_6.iloc[:,0]
            np_DWFD_20_V = np.array(df_DWFD_20_V)#np.ndarray()
            #print(df_DWFD_0_V)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_20_T_V_list = np_DWFD_20_V.tolist()#list
            
            
            #获取待写入报告的温度数据
            df_DWFD_20_T = df_DWFD_20_6.iloc[:,3]
            np_DWFD_20_T = np.array(df_DWFD_20_T)#np.ndarray()
            #print(df_DWFD_0_T)
            #将获取的温度数据转换为列表，方便后期写入excel
            DWFD_20_T_T_list = np_DWFD_20_T.tolist()#list
            #取第一个温度值
            DWFD_20_T_T_1 = DWFD_20_T_T_list[0]
            #取最后一个温度值，计算放电温升
            DWFD_20_T_T_2 = DWFD_20_T_T_list[-1]
            DWFD_20_T_rise = DWFD_20_T_T_2 - DWFD_20_T_T_1
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            DWFD_20_T_T_2_list = []
            for i_DWFD_20_T_T in range(0,DWFD_20_T_C_list_len):
                #遍历温度列表，减去初始温度，得到温升值，温升值用round取两位小数
                DWFD_20_T_T_2_list_i = round(DWFD_20_T_T_list[i_DWFD_20_T_T] - DWFD_20_T_T_1, 2)
                DWFD_20_T_T_2_list.append(DWFD_20_T_T_2_list_i)
            #print(DWFD_0_T_T_2_list)
            
           
            #打开报告,写入数据
            #准备打开报告，写入-10℃放电容量数据
            wb_DWFD_20_T = openpyxl.load_workbook("%s" % test_file_name_1)
            ws_DWFD_20_T = wb_DWFD_20_T["不同温度放电"]
            #填入序号、条码、容量、温升数据
            #k_SWFD_3 = counter_SWFD +27
            #k_DWFD_20_T_4 = counter_DWFD_20_T + 9 + (counter_DWFD_20_T - 1) * 28  #10lie
            #k_DWFD_20_T_5 = counter_DWFD_20_T + 9 + (counter_DWFD_20_T - 1) * 28 + 28  #38列
            #指定写入绘图表头的初始列索引
            #k_DWFD_0_T_6 = get_column_letter(k_SWFD_4)
            #k_DWFD_0_T_7 = get_column_letter(k_SWFD_5)
            #指定写入-10℃数据的初始列索引
            
            k_DWFD_20_T_1 = counter_DWFD_20_T + 12 + (counter_DWFD_20_T - 1) * 27 + 12   #25列
            
            #写入数据
            #ws_DWFD_0_T.cell(row = 1,column = k_DWFD_0_T_4, value = DWFD_0_T_cell_num)
            
            #if ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).value == DWFD_0_T_cell_num:
                
            for i_DWFD_20_T_5 in range(1, 5):
                #for col in range("%s" % row_DCR, "%s" % row_DCR)
                DWFD_20_T_col_list = ["-20℃容量（Ah）", "-20℃电压（V）", "-20℃温度（℃）", "-20℃温升（℃）"]
                ws_DWFD_20_T.cell(row = 2,column = k_DWFD_20_T_1+i_DWFD_20_T_5, value = DWFD_20_T_col_list[i_DWFD_20_T_5 -1])

            for j_DWFD_20_T in range(1, DWFD_20_T_C_list_len+1):
                #写入容量、电压、温度、温升四列数据
                ws_DWFD_20_T.cell(row = j_DWFD_20_T+2, column = k_DWFD_20_T_1+1, value = DWFD_20_T_C_list[j_DWFD_20_T-1]) 
                ws_DWFD_20_T.cell(row = j_DWFD_20_T+2, column = k_DWFD_20_T_1+2, value = DWFD_20_T_V_list[j_DWFD_20_T-1])
                ws_DWFD_20_T.cell(row = j_DWFD_20_T+2, column = k_DWFD_20_T_1+3, value = DWFD_20_T_T_list[j_DWFD_20_T-1])
                ws_DWFD_20_T.cell(row = j_DWFD_20_T+2, column = k_DWFD_20_T_1+4, value = DWFD_20_T_T_2_list[j_DWFD_20_T-1])
                
            #写入容量表  
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_20_T.cell(row = counter_DWFD_20_T +2,column = 7, value = DWFD_20_T_C)
            #写入比率表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            
            ws_DWFD_20_T.cell(row = counter_DWFD_20_T +15,column = 7, value = "=G%s/D%s" % (counter_DWFD_20_T +2, counter_DWFD_20_T +2))
            #写入温升表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_20_T.cell(row = counter_DWFD_20_T +27,column = 7, value = DWFD_20_T_rise)
            #else:
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 5, value = "请检查是否缺数据或者条码错误")

           
                
            #填入电压数据    
            #for j in range(1, DCR_V_10C_list_len+1):
                #ws_DCR.cell(row = j+12, column = counter_DCR, value = DCR_V_10C_list[j-1])
            #ws_DCR.cell(row = counter_DCR +2,column = 1).alignment = Alignment(horizontal='right', vertical='center')
            
            #计数器+1
            counter_DWFD_20_T += 1

            #print(ws_SWFD)
            wb_DWFD_20_T.save("%s" % test_file_name_1)
            print("1#" * 30)
            print("1#" * 30)
           
        #判断不同温度放电\-30℃
        elif file_name_dq == "不同温度放电\\-30℃":
            print("准备调用不同温度放电-30℃.py")
            #判断不同温度放电\-30℃
            DWFD_30_T_cell_num = file_name_1[ : TM_位数]
            print(DWFD_30_T_cell_num)
            #打开数据
            df_DWFD_30_1 = pd.ExcelFile(file_path)
            DWFD_30_sheetnames_list = df_DWFD_30_1.sheet_names  # 获取工作表名称
            DWFD_30_sheetname = DWFD_30_sheetnames_list[1]
            #读取详细数据文件
            df_DWFD_30_2 = pd.read_excel(file_path, sheet_name = DWFD_30_sheetname)
            df_DWFD_30_3 = df_DWFD_30_2.iloc[:, [5, 6, 8, 13]]#索引切片，获取需要的电压、放电容量、温升数据
            df_DWFD_30_4 = df_DWFD_30_3[df_DWFD_30_3["Current(A)"] <= -9]
            
            #获取开头静置数据行数
            df_DWFD_30_5 = df_DWFD_30_3[df_DWFD_30_3["Discharge_Capacity(Ah)"] <= 0]
            #print(df_DWFD_0_5)
            #获取容量一列数据
            df_DWFD_30_C_1 = df_DWFD_30_5.iloc[ : , 2]
            np_DWFD_30_C_1 = np.array(df_DWFD_30_C_1)#np.ndarray()
            DWFD_30_C_1_list = np_DWFD_30_C_1.tolist()
            #取静置最后一秒数据索引号
            num_DWFD_30_df_1 = len(DWFD_30_C_1_list)-1
            #取电压数据一列
            df_DWFD_30_V_1 = df_DWFD_30_4.iloc[ : , 0] 
            #获取低温放电数据行数
            np_DWFD_30_V_1 = np.array(df_DWFD_30_V_1)#np.ndarray()
            DWFD_30_V_1_list = np_DWFD_30_V_1.tolist()
            num_DWFD_30_df = len(DWFD_30_V_1_list)+1
            #print(num_DWFD_0_df_1)
            #print(num_DWFD_0_df)
            
            #索引切片截取所需数据段(行、列)
            df_DWFD_30_6 = df_DWFD_30_3.iloc[num_DWFD_30_df_1:num_DWFD_30_df+num_DWFD_30_df_1,:]
            #print(df_DWFD_0_6)
            
            #获取待写入报告的容量数据
            df_DWFD_30_C = df_DWFD_30_6.iloc[:,2]
            np_DWFD_30_C = np.array(df_DWFD_30_C)#np.ndarray()
            #print(df_DWFD_0_C)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_30_T_C_list = np_DWFD_30_C.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            DWFD_30_T_C = DWFD_30_T_C_list[-1]
            #数据列表长度,后期向报告单元格写入这一列数据需要用到，作为遍历循环的上限值
            DWFD_30_T_C_list_len = len(DWFD_30_T_C_list)
            #print(DWFD_0_T_C_list_len)
            
            #获取待写入报告的电压数据
            df_DWFD_30_V = df_DWFD_30_6.iloc[:,0]
            np_DWFD_30_V = np.array(df_DWFD_30_V)#np.ndarray()
            #print(df_DWFD_0_V)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_30_T_V_list = np_DWFD_30_V.tolist()#list
            
            
            #获取待写入报告的温度数据
            df_DWFD_30_T = df_DWFD_30_6.iloc[:,3]
            np_DWFD_30_T = np.array(df_DWFD_30_T)#np.ndarray()
            #print(df_DWFD_0_T)
            #将获取的温度数据转换为列表，方便后期写入excel
            DWFD_30_T_T_list = np_DWFD_30_T.tolist()#list
            #取第一个温度值
            DWFD_30_T_T_1 = DWFD_30_T_T_list[0]
            #取最后一个温度值，计算放电温升
            DWFD_30_T_T_2 = DWFD_30_T_T_list[-1]
            DWFD_30_T_rise = DWFD_30_T_T_2 - DWFD_30_T_T_1
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            DWFD_30_T_T_2_list = []
            for i_DWFD_30_T_T in range(0,DWFD_30_T_C_list_len):
                #遍历温度列表，减去初始温度，得到温升值，温升值用round取两位小数
                DWFD_30_T_T_2_list_i = round(DWFD_30_T_T_list[i_DWFD_30_T_T] - DWFD_30_T_T_1, 2)
                DWFD_30_T_T_2_list.append(DWFD_30_T_T_2_list_i)
            #print(DWFD_0_T_T_2_list)
            
           
            #打开报告,写入数据
            #准备打开报告，写入-10℃放电容量数据
            wb_DWFD_30_T = openpyxl.load_workbook("%s" % test_file_name_1)
            ws_DWFD_30_T = wb_DWFD_30_T["不同温度放电"]
            #填入序号、条码、容量、温升数据
            #k_SWFD_3 = counter_SWFD +27
            k_DWFD_30_T_4 = counter_DWFD_30_T + 9 + (counter_DWFD_30_T - 1) * 28  #10lie
            k_DWFD_30_T_5 = counter_DWFD_30_T + 9 + (counter_DWFD_30_T - 1) * 28 + 28  #38列
            #指定写入绘图表头的初始列索引
            #k_DWFD_0_T_6 = get_column_letter(k_SWFD_4)
            #k_DWFD_0_T_7 = get_column_letter(k_SWFD_5)
            #指定写入-10℃数据的初始列索引
            
            k_DWFD_30_T_1 = counter_DWFD_30_T + 12 + (counter_DWFD_30_T - 1) * 27 + 16   #29列
            
            #写入数据
            #ws_DWFD_0_T.cell(row = 1,column = k_DWFD_0_T_4, value = DWFD_0_T_cell_num)
            
            #if ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).value == DWFD_0_T_cell_num:
                
            for i_DWFD_30_T_5 in range(1, 5):
                #for col in range("%s" % row_DCR, "%s" % row_DCR)
                DWFD_30_T_col_list = ["-30℃容量（Ah）", "-30℃电压（V）", "-30℃温度（℃）", "-30℃温升（℃）"]
                ws_DWFD_30_T.cell(row = 2,column = k_DWFD_30_T_1+i_DWFD_30_T_5, value = DWFD_30_T_col_list[i_DWFD_30_T_5 -1])

            for j_DWFD_30_T in range(1, DWFD_30_T_C_list_len+1):
                #写入容量、电压、温度、温升四列数据
                ws_DWFD_30_T.cell(row = j_DWFD_30_T+2, column = k_DWFD_30_T_1+1, value = DWFD_30_T_C_list[j_DWFD_30_T-1]) 
                ws_DWFD_30_T.cell(row = j_DWFD_30_T+2, column = k_DWFD_30_T_1+2, value = DWFD_30_T_V_list[j_DWFD_30_T-1])
                ws_DWFD_30_T.cell(row = j_DWFD_30_T+2, column = k_DWFD_30_T_1+3, value = DWFD_30_T_T_list[j_DWFD_30_T-1])
                ws_DWFD_30_T.cell(row = j_DWFD_30_T+2, column = k_DWFD_30_T_1+4, value = DWFD_30_T_T_2_list[j_DWFD_30_T-1])
                
            #写入容量表  
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_30_T.cell(row = counter_DWFD_30_T +2,column = 8, value = DWFD_30_T_C)
            #写入比率表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            
            ws_DWFD_30_T.cell(row = counter_DWFD_30_T +15,column = 8, value = "=H%s/D%s" % (counter_DWFD_30_T +2, counter_DWFD_30_T +2))
            #写入温升表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_30_T.cell(row = counter_DWFD_30_T +27,column = 8, value = DWFD_30_T_rise)
            #else:
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 5, value = "请检查是否缺数据或者条码错误")

           
                
            #填入电压数据    
            #for j in range(1, DCR_V_10C_list_len+1):
                #ws_DCR.cell(row = j+12, column = counter_DCR, value = DCR_V_10C_list[j-1])
            #ws_DCR.cell(row = counter_DCR +2,column = 1).alignment = Alignment(horizontal='right', vertical='center')
            
            #计数器+1
            counter_DWFD_30_T += 1

            #print(ws_SWFD)
            wb_DWFD_30_T.save("%s" % test_file_name_1)
            print("1#" * 30)
            print("1#" * 30)
            
        #判断不同温度放电\-40℃
        elif file_name_dq == "不同温度放电\\-40℃":
            print("准备调用不同温度放电-40℃.py")
            #判断不同温度放电\-40℃
            DWFD_40_T_cell_num = file_name_1[ : TM_位数]
            print(DWFD_40_T_cell_num)
            #打开数据
            df_DWFD_40_1 = pd.ExcelFile(file_path)
            DWFD_40_sheetnames_list = df_DWFD_40_1.sheet_names  # 获取工作表名称
            DWFD_40_sheetname = DWFD_40_sheetnames_list[1]
            #读取详细数据文件
            df_DWFD_40_2 = pd.read_excel(file_path, sheet_name = DWFD_40_sheetname)
            df_DWFD_40_3 = df_DWFD_40_2.iloc[:, [5, 6, 8, 13]]#索引切片，获取需要的电压、放电容量、温升数据
            df_DWFD_40_4 = df_DWFD_40_3[df_DWFD_40_3["Current(A)"] <= -9]
            
            #获取开头静置数据行数
            df_DWFD_40_5 = df_DWFD_40_3[df_DWFD_40_3["Discharge_Capacity(Ah)"] <= 0]
            #print(df_DWFD_0_5)
            #获取容量一列数据
            df_DWFD_40_C_1 = df_DWFD_40_5.iloc[ : , 2]
            np_DWFD_40_C_1 = np.array(df_DWFD_40_C_1)#np.ndarray()
            DWFD_40_C_1_list = np_DWFD_40_C_1.tolist()
            #取静置最后一秒数据索引号
            num_DWFD_40_df_1 = len(DWFD_40_C_1_list)-1
            #取电压数据一列
            df_DWFD_40_V_1 = df_DWFD_40_4.iloc[ : , 0] 
            #获取低温放电数据行数
            np_DWFD_40_V_1 = np.array(df_DWFD_40_V_1)#np.ndarray()
            DWFD_40_V_1_list = np_DWFD_40_V_1.tolist()
            num_DWFD_40_df = len(DWFD_40_V_1_list)+1
            #print(num_DWFD_0_df_1)
            #print(num_DWFD_0_df)
            
            #索引切片截取所需数据段(行、列)
            df_DWFD_40_6 = df_DWFD_40_3.iloc[num_DWFD_40_df_1:num_DWFD_40_df+num_DWFD_40_df_1,:]
            #print(df_DWFD_0_6)
            
            #获取待写入报告的容量数据
            df_DWFD_40_C = df_DWFD_40_6.iloc[:,2]
            np_DWFD_40_C = np.array(df_DWFD_40_C)#np.ndarray()
            #print(df_DWFD_0_C)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_40_T_C_list = np_DWFD_40_C.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            DWFD_40_T_C = DWFD_40_T_C_list[-1]
            #数据列表长度,后期向报告单元格写入这一列数据需要用到，作为遍历循环的上限值
            DWFD_40_T_C_list_len = len(DWFD_40_T_C_list)
            #print(DWFD_0_T_C_list_len)
            
            #获取待写入报告的电压数据
            df_DWFD_40_V = df_DWFD_40_6.iloc[:,0]
            np_DWFD_40_V = np.array(df_DWFD_40_V)#np.ndarray()
            #print(df_DWFD_0_V)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_40_T_V_list = np_DWFD_40_V.tolist()#list
            
            
            #获取待写入报告的温度数据
            df_DWFD_40_T = df_DWFD_40_6.iloc[:,3]
            np_DWFD_40_T = np.array(df_DWFD_40_T)#np.ndarray()
            #print(df_DWFD_0_T)
            #将获取的温度数据转换为列表，方便后期写入excel
            DWFD_40_T_T_list = np_DWFD_40_T.tolist()#list
            #取第一个温度值
            DWFD_40_T_T_1 = DWFD_40_T_T_list[0]
            #取最后一个温度值，计算放电温升
            DWFD_40_T_T_2 = DWFD_40_T_T_list[-1]
            DWFD_40_T_rise = DWFD_40_T_T_2 - DWFD_40_T_T_1
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            DWFD_40_T_T_2_list = []
            for i_DWFD_40_T_T in range(0,DWFD_40_T_C_list_len):
                #遍历温度列表，减去初始温度，得到温升值，温升值用round取两位小数
                DWFD_40_T_T_2_list_i = round(DWFD_40_T_T_list[i_DWFD_40_T_T] - DWFD_40_T_T_1, 2)
                DWFD_40_T_T_2_list.append(DWFD_40_T_T_2_list_i)
            #print(DWFD_0_T_T_2_list)
            
           
            #打开报告,写入数据
            #准备打开报告，写入-10℃放电容量数据
            wb_DWFD_40_T = openpyxl.load_workbook("%s" % test_file_name_1)
            ws_DWFD_40_T = wb_DWFD_40_T["不同温度放电"]
            #填入序号、条码、容量、温升数据
            #k_SWFD_3 = counter_SWFD +27
            #k_DWFD_40_T_4 = counter_DWFD_40_T + 9 + (counter_DWFD_40_T - 1) * 28  #10lie
            #k_DWFD_40_T_5 = counter_DWFD_40_T + 9 + (counter_DWFD_40_T - 1) * 28 + 28  #38列
            #指定写入绘图表头的初始列索引
            #k_DWFD_0_T_6 = get_column_letter(k_SWFD_4)
            #k_DWFD_0_T_7 = get_column_letter(k_SWFD_5)
            #指定写入-10℃数据的初始列索引
            
            k_DWFD_40_T_1 = counter_DWFD_40_T + 12 + (counter_DWFD_40_T - 1) * 27 + 20   #33列
            
            #写入数据
            #ws_DWFD_0_T.cell(row = 1,column = k_DWFD_0_T_4, value = DWFD_0_T_cell_num)
            
            #if ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).value == DWFD_0_T_cell_num:
                
            for i_DWFD_40_T_5 in range(1, 5):
                #for col in range("%s" % row_DCR, "%s" % row_DCR)
                DWFD_40_T_col_list = ["-40℃容量（Ah）", "-40℃电压（V）", "-40℃温度（℃）", "-40℃温升（℃）"]
                ws_DWFD_40_T.cell(row = 2,column = k_DWFD_40_T_1+i_DWFD_40_T_5, value = DWFD_40_T_col_list[i_DWFD_40_T_5 -1])

            for j_DWFD_40_T in range(1, DWFD_40_T_C_list_len+1):
                #写入容量、电压、温度、温升四列数据
                ws_DWFD_40_T.cell(row = j_DWFD_40_T+2, column = k_DWFD_40_T_1+1, value = DWFD_40_T_C_list[j_DWFD_40_T-1]) 
                ws_DWFD_40_T.cell(row = j_DWFD_40_T+2, column = k_DWFD_40_T_1+2, value = DWFD_40_T_V_list[j_DWFD_40_T-1])
                ws_DWFD_40_T.cell(row = j_DWFD_40_T+2, column = k_DWFD_40_T_1+3, value = DWFD_40_T_T_list[j_DWFD_40_T-1])
                ws_DWFD_40_T.cell(row = j_DWFD_40_T+2, column = k_DWFD_40_T_1+4, value = DWFD_40_T_T_2_list[j_DWFD_40_T-1])
                
            #写入容量表  
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_40_T.cell(row = counter_DWFD_40_T +2,column = 9, value = DWFD_40_T_C)
            #写入比率表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            
            ws_DWFD_40_T.cell(row = counter_DWFD_40_T +15,column = 9, value = "=I%s/D%s" % (counter_DWFD_40_T +2, counter_DWFD_40_T +2))
            #写入温升表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_40_T.cell(row = counter_DWFD_40_T +27,column = 9, value = DWFD_40_T_rise)
            #else:
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 5, value = "请检查是否缺数据或者条码错误")

           
                
            #填入电压数据    
            #for j in range(1, DCR_V_10C_list_len+1):
                #ws_DCR.cell(row = j+12, column = counter_DCR, value = DCR_V_10C_list[j-1])
            #ws_DCR.cell(row = counter_DCR +2,column = 1).alignment = Alignment(horizontal='right', vertical='center')
            
            #计数器+1
            counter_DWFD_40_T += 1

            #print(ws_SWFD)
            wb_DWFD_40_T.save("%s" % test_file_name_1)
            print("1#" * 30)
            print("1#" * 30)
           
        
        #判断不同温度放电\55℃
        elif file_name_dq == "不同温度放电\\55℃":
            print("准备调用不同温度放电55℃.py")
            #判断不同温度放电\55℃
            DWFD_55_T_cell_num = file_name_1[ : TM_位数]
            print(DWFD_55_T_cell_num)
            #打开数据
            df_DWFD_55_1 = pd.ExcelFile(file_path)
            DWFD_55_sheetnames_list = df_DWFD_55_1.sheet_names  # 获取工作表名称
            DWFD_55_sheetname = DWFD_55_sheetnames_list[1]
            #读取详细数据文件
            df_DWFD_55_2 = pd.read_excel(file_path, sheet_name = DWFD_55_sheetname)
            df_DWFD_55_3 = df_DWFD_55_2.iloc[:, [5, 6, 8, 13]]#索引切片，获取需要的电压、放电容量、温升数据
            df_DWFD_55_4 = df_DWFD_55_3[df_DWFD_55_3["Current(A)"] <= -9]
            
            #获取开头静置数据行数
            df_DWFD_55_5 = df_DWFD_55_3[df_DWFD_55_3["Discharge_Capacity(Ah)"] <= 0]
            #print(df_DWFD_0_5)
            #获取容量一列数据
            df_DWFD_55_C_1 = df_DWFD_55_5.iloc[ : , 2]
            np_DWFD_55_C_1 = np.array(df_DWFD_55_C_1)#np.ndarray()
            DWFD_55_C_1_list = np_DWFD_55_C_1.tolist()
            #取静置最后一秒数据索引号
            num_DWFD_55_df_1 = len(DWFD_55_C_1_list)-1
            #取电压数据一列
            df_DWFD_55_V_1 = df_DWFD_55_4.iloc[ : , 0] 
            #获取低温放电数据行数
            np_DWFD_55_V_1 = np.array(df_DWFD_55_V_1)#np.ndarray()
            DWFD_55_V_1_list = np_DWFD_55_V_1.tolist()
            num_DWFD_55_df = len(DWFD_55_V_1_list)+1
            #print(num_DWFD_0_df_1)
            #print(num_DWFD_0_df)
            
            #索引切片截取所需数据段(行、列)
            df_DWFD_55_6 = df_DWFD_55_3.iloc[num_DWFD_55_df_1:num_DWFD_55_df+num_DWFD_55_df_1,:]
            #print(df_DWFD_0_6)
            
            #获取待写入报告的容量数据
            df_DWFD_55_C = df_DWFD_55_6.iloc[:,2]
            np_DWFD_55_C = np.array(df_DWFD_55_C)#np.ndarray()
            #print(df_DWFD_0_C)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_55_T_C_list = np_DWFD_55_C.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            DWFD_55_T_C = DWFD_55_T_C_list[-1]
            #数据列表长度,后期向报告单元格写入这一列数据需要用到，作为遍历循环的上限值
            DWFD_55_T_C_list_len = len(DWFD_55_T_C_list)
            #print(DWFD_0_T_C_list_len)
            
            #获取待写入报告的电压数据
            df_DWFD_55_V = df_DWFD_55_6.iloc[:,0]
            np_DWFD_55_V = np.array(df_DWFD_55_V)#np.ndarray()
            #print(df_DWFD_0_V)
            #将获取的容量数据转换为列表，方便后期写入excel
            DWFD_55_T_V_list = np_DWFD_55_V.tolist()#list
            
            
            #获取待写入报告的温度数据
            df_DWFD_55_T = df_DWFD_55_6.iloc[:,3]
            np_DWFD_55_T = np.array(df_DWFD_55_T)#np.ndarray()
            #print(df_DWFD_0_T)
            #将获取的温度数据转换为列表，方便后期写入excel
            DWFD_55_T_T_list = np_DWFD_55_T.tolist()#list
            #取第一个温度值
            DWFD_55_T_T_1 = DWFD_55_T_T_list[0]
            #取最后一个温度值，计算放电温升
            DWFD_55_T_T_2 = DWFD_55_T_T_list[-1]
            DWFD_55_T_rise = DWFD_55_T_T_2 - DWFD_55_T_T_1
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            DWFD_55_T_T_2_list = []
            for i_DWFD_55_T_T in range(0,DWFD_55_T_C_list_len):
                #遍历温度列表，减去初始温度，得到温升值，温升值用round取两位小数
                DWFD_55_T_T_2_list_i = round(DWFD_55_T_T_list[i_DWFD_55_T_T] - DWFD_55_T_T_1, 2)
                DWFD_55_T_T_2_list.append(DWFD_55_T_T_2_list_i)
            #print(DWFD_0_T_T_2_list)
            
           
            #打开报告,写入数据
            #准备打开报告，写入-10℃放电容量数据
            wb_DWFD_55_T = openpyxl.load_workbook("%s" % test_file_name_1)
            ws_DWFD_55_T = wb_DWFD_55_T["不同温度放电"]
            #填入序号、条码、容量、温升数据
            #k_SWFD_3 = counter_SWFD +27
            #k_DWFD_55_T_4 = counter_DWFD_55_T + 8 + (counter_DWFD_55_T - 1) * 28  #9lie
            #k_DWFD_55_T_5 = counter_DWFD_55_T + 8 + (counter_DWFD_55_T - 1) * 28  #9列
            #指定写入绘图表头的初始列索引
            #k_DWFD_0_T_6 = get_column_letter(k_SWFD_4)
            #k_DWFD_0_T_7 = get_column_letter(k_SWFD_5)
            #指定写入-10℃数据的初始列索引
            
            k_DWFD_55_T_1 = counter_DWFD_55_T + 8 + (counter_DWFD_55_T - 1) * 27   #9列
            
            #写入数据
            #ws_DWFD_0_T.cell(row = 1,column = k_DWFD_0_T_4, value = DWFD_0_T_cell_num)
            
            #if ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).value == DWFD_0_T_cell_num:
                
            for i_DWFD_55_T_5 in range(1, 5):
                #for col in range("%s" % row_DCR, "%s" % row_DCR)
                DWFD_55_T_col_list = ["55℃容量（Ah）", "55℃电压（V）", "55℃温度（℃）", "55℃温升（℃）"]
                ws_DWFD_55_T.cell(row = 2,column = k_DWFD_55_T_1+i_DWFD_55_T_5, value = DWFD_55_T_col_list[i_DWFD_55_T_5 -1])

            for j_DWFD_55_T in range(1, DWFD_55_T_C_list_len+1):
                #写入容量、电压、温度、温升四列数据
                ws_DWFD_55_T.cell(row = j_DWFD_55_T+2, column = k_DWFD_55_T_1+1, value = DWFD_55_T_C_list[j_DWFD_55_T-1]) 
                ws_DWFD_55_T.cell(row = j_DWFD_55_T+2, column = k_DWFD_55_T_1+2, value = DWFD_55_T_V_list[j_DWFD_55_T-1])
                ws_DWFD_55_T.cell(row = j_DWFD_55_T+2, column = k_DWFD_55_T_1+3, value = DWFD_55_T_T_list[j_DWFD_55_T-1])
                ws_DWFD_55_T.cell(row = j_DWFD_55_T+2, column = k_DWFD_55_T_1+4, value = DWFD_55_T_T_2_list[j_DWFD_55_T-1])
                
            #写入容量表  
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_55_T.cell(row = counter_DWFD_55_T +2,column = 3, value = DWFD_55_T_C)
            #写入比率表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            
            ws_DWFD_55_T.cell(row = counter_DWFD_55_T +15,column = 3, value = "=C%s/D%s" % (counter_DWFD_55_T +2, counter_DWFD_55_T +2))
            #写入温升表    
            #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_DWFD_55_T.cell(row = counter_DWFD_55_T +27,column = 3, value = DWFD_55_T_rise)
            #else:
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +2,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +15,column = 5, value = "请检查是否缺数据或者条码错误")
                #ws_DWFD_0_T.cell(row = counter_DWFD_0_T +27,column = 5, value = "请检查是否缺数据或者条码错误")

           
                
            #填入电压数据    
            #for j in range(1, DCR_V_10C_list_len+1):
                #ws_DCR.cell(row = j+12, column = counter_DCR, value = DCR_V_10C_list[j-1])
            #ws_DCR.cell(row = counter_DCR +2,column = 1).alignment = Alignment(horizontal='right', vertical='center')
            
            #计数器+1
            counter_DWFD_55_T += 1

            #print(ws_SWFD)
            wb_DWFD_55_T.save("%s" % test_file_name_1)
            print("1#" * 30)
            print("1#" * 30)
           
        
        #判断文件夹是否是倍率放电
        elif file_name_dq == "倍率放电":
            print("准备处理倍率放电数据")
            
            BLFD_cell_num = file_name_1[ : TM_位数]
            print(BLFD_cell_num)
            
            #输入工作簿名字，准备赋值给DataFrame
            #获取循环数据工作簿名字
            sheetName_BLFD_C = "cycle"
            #获取详细数据工作簿名字
            sheetName_BLFD_V = "record"
            #获取温度数据工作簿名字
            sheetName_BLFD_T = "auxTemperature"
            
            #用DataFrame取循环数据
            df_BLFD_1_1 = pd.read_excel(file_path, sheet_name = sheetName_BLFD_C)
            #print(df_SWFD_1)
            df_BLFD_2 = df_BLFD_1_1.iloc[:, [2]]#索引切片（第3列数据），获取放电容量数据
            #print(df_SWFD_2)
            df_BLFD_3 = df_BLFD_2[df_BLFD_2["放电容量(Ah)"] > 8]#索引切片（容量大于8的数据），获取初始3次容量数据
            #print(df_SWFD_3)
            #np_SWFD_C_25度 = np.array(df_SWFD_3)#将二维DataFrame数组通过np.ndarray()转换成一维数据
            #将获取的容量数据转换为列表，方便计算初始容量和后期写入excel
            #SWFD_C_25度_list = np_SWFD_C_25度.tolist()#list
            #print(SWFD_C_25度_list)
            #三次容量求和
            BLFD_C1 = df_BLFD_3.at[1, "放电容量(Ah)"]
            BLFD_C2 = df_BLFD_3.at[2, "放电容量(Ah)"]
            BLFD_C3 = df_BLFD_3.at[3, "放电容量(Ah)"]
            
            #print(SWFD_C1, SWFD_C2, SWFD_C3)
            #计算不同温度放电初始容量
            BLFD_C = round((BLFD_C1 + BLFD_C2+ BLFD_C3)/3, 4)
            #print(SWFD_C)
            
            #用DataFrame取温度数据
            df_BLFD_T_1 = pd.read_excel(file_path, sheet_name = sheetName_BLFD_T)
            df_BLFD_T_2 = df_BLFD_T_1.iloc[:, [0, 4]]
            #print(df_SWFD_T_2)
            #用DataFrame取电压数据
            df_BLFD_V_1 = pd.read_excel(file_path, sheet_name = sheetName_BLFD_V)
            df_BLFD_V_2 = df_BLFD_V_1.iloc[:, 1:8]
            #print(df_BLFD_V_2)
            
            #将两张表数据汇总到一起进行电压和温度数据的提取
            df_BLFD_1 = pd.merge(df_BLFD_V_2, df_BLFD_T_2, on='记录序号')
            #print(df_BLFD_1)
            
            
            #获取40C放电数据,和数据长度
            df_BLFD_40C_1 = df_BLFD_1[df_BLFD_1["电流(A)"] <= -350]
            len(df_BLFD_40C_1)
            #print(len(df_BLFD_10C_1))
            #print(df_BLFD_10C_1)
            #获取40C第一个数据序号
            BLFD_No_40C_1 = df_BLFD_40C_1.iloc[0, 2] - 1
            #获取40C待写入的数据
            df_BLFD_40C = df_BLFD_1.iloc[BLFD_No_40C_1 : BLFD_No_40C_1 + len(df_BLFD_40C_1), [4, 6, 7]]
            #print(df_BLFD_40C)
            #print(BLFD_No_40C_1)
            #print(BLFD_No_40C_1 + len(df_BLFD_10C_1))
            
            #取电压数据一列
            df_BLFD_40C_V_1 = df_BLFD_40C.iloc[ : , 0] 
            np_BLFD_40C_V_1 = np.array(df_BLFD_40C_V_1)#np.ndarray()
            #将获取的电压数据转换为列表，方便后期写入excel
            BLFD_40C_V_1_list = np_BLFD_40C_V_1.tolist()#list
            #print(BLFD_40C_V_1_list)
            
            #取容量数据一列
            df_BLFD_40C_C_1 = df_BLFD_40C.iloc[ : , 1] 
            np_BLFD_40C_C_1 = np.array(df_BLFD_40C_C_1)#np.ndarray()
            #将获取的容量数据转换为列表，方便后期写入excel
            BLFD_40C_C_1_list = np_BLFD_40C_C_1.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            BLFD_40C_C_1 = BLFD_40C_C_1_list[-1]
            #数据列表长度
            BLFD_40C_C_1_list_len = len(BLFD_40C_C_1_list)
            #print(SWFD_C_1_list_len)
            #print(BLFD_40C_C_1_list)
            
            #取温度数据一列
            df_BLFD_40C_T_1 = df_BLFD_40C.iloc[ : , 2] 
            np_BLFD_40C_T_1 = np.array(df_BLFD_40C_T_1)#np.ndarray()
            #将获取的温度数据转换为列表，方便后期写入excel
            BLFD_40C_T_1_list = np_BLFD_40C_T_1.tolist()#list
            #取第一个和最后一个温度值作为计算当前温度下放电温升的数据
            BLFD_40C_T_1 = BLFD_40C_T_1_list[0]
            BLFD_40C_T_2 = BLFD_40C_T_1_list[-1]
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            BLFD_40C_T_2_list = []
            for i_BLFD_40C_T in range(0,BLFD_40C_C_1_list_len):
                BLFD_40C_T_2_list_i = BLFD_40C_T_1_list[i_BLFD_40C_T] - BLFD_40C_T_1
                BLFD_40C_T_2_list.append(BLFD_40C_T_2_list_i)
            #print(SWFD_T_2_list)
            #计算当前温度放电温升
            BLFD_40C_T_rise = round(BLFD_40C_T_2 - BLFD_40C_T_1, 2)
            #print(BLFD_40C_T_2_list)
            #print(BLFD_40C_T_1_list)
            
            
            #获取30C放电数据,和数据长度
            df_BLFD_30C_1_1 = df_BLFD_1[df_BLFD_1["电流(A)"] <= -250]
            df_BLFD_30C_1 = df_BLFD_30C_1_1[df_BLFD_30C_1_1["电流(A)"] > -350]
            len(df_BLFD_30C_1)
            #print(len(df_BLFD_10C_1))
            #print(df_BLFD_30C_1)
            #获取40C第一个数据序号
            BLFD_No_30C_1 = df_BLFD_30C_1.iloc[0, 2] - 1
            #获取40C待写入的数据
            df_BLFD_30C = df_BLFD_1.iloc[BLFD_No_30C_1 : BLFD_No_30C_1 + len(df_BLFD_30C_1), [4, 6, 7]]
            #print(df_BLFD_40C)
            #print(BLFD_No_40C_1)
            #print(BLFD_No_40C_1 + len(df_BLFD_10C_1))
            
            #取电压数据一列
            df_BLFD_30C_V_1 = df_BLFD_30C.iloc[ : , 0] 
            np_BLFD_30C_V_1 = np.array(df_BLFD_30C_V_1)#np.ndarray()
            #将获取的电压数据转换为列表，方便后期写入excel
            BLFD_30C_V_1_list = np_BLFD_30C_V_1.tolist()#list
            #print(BLFD_40C_V_1_list)
            
            #取容量数据一列
            df_BLFD_30C_C_1 = df_BLFD_30C.iloc[ : , 1] 
            np_BLFD_30C_C_1 = np.array(df_BLFD_30C_C_1)#np.ndarray()
            #将获取的容量数据转换为列表，方便后期写入excel
            BLFD_30C_C_1_list = np_BLFD_30C_C_1.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            BLFD_30C_C_1 = BLFD_30C_C_1_list[-1]
            #数据列表长度
            BLFD_30C_C_1_list_len = len(BLFD_30C_C_1_list)
            #print(SWFD_C_1_list_len)
            #print(BLFD_40C_C_1_list)
            
            #取温度数据一列
            df_BLFD_30C_T_1 = df_BLFD_30C.iloc[ : , 2] 
            np_BLFD_30C_T_1 = np.array(df_BLFD_30C_T_1)#np.ndarray()
            #将获取的温度数据转换为列表，方便后期写入excel
            BLFD_30C_T_1_list = np_BLFD_30C_T_1.tolist()#list
            #取第一个和最后一个温度值作为计算当前温度下放电温升的数据
            BLFD_30C_T_1 = BLFD_30C_T_1_list[0]
            BLFD_30C_T_2 = BLFD_30C_T_1_list[-1]
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            BLFD_30C_T_2_list = []
            for i_BLFD_30C_T in range(0,BLFD_30C_C_1_list_len):
                BLFD_30C_T_2_list_i = BLFD_30C_T_1_list[i_BLFD_30C_T] - BLFD_30C_T_1
                BLFD_30C_T_2_list.append(BLFD_30C_T_2_list_i)
            #print(SWFD_T_2_list)
            #计算当前温度放电温升
            BLFD_30C_T_rise = round(BLFD_30C_T_2 - BLFD_30C_T_1, 2)
            #print(BLFD_40C_T_2_list)
            #print(BLFD_40C_T_1_list)
            
            
            #获取20C放电数据,和数据长度
            df_BLFD_20C_1_1 = df_BLFD_1[df_BLFD_1["电流(A)"] <= -150]
            df_BLFD_20C_1 = df_BLFD_20C_1_1[df_BLFD_20C_1_1["电流(A)"] > -250]
            len(df_BLFD_20C_1)
            #print(len(df_BLFD_10C_1))
            #print(df_BLFD_20C_1)
            #获取40C第一个数据序号
            BLFD_No_20C_1 = df_BLFD_20C_1.iloc[0, 2] - 1
            #获取40C待写入的数据
            df_BLFD_20C = df_BLFD_1.iloc[BLFD_No_20C_1 : BLFD_No_20C_1 + len(df_BLFD_20C_1), [4, 6, 7]]
            #print(df_BLFD_40C)
            #print(BLFD_No_40C_1)
            #print(BLFD_No_40C_1 + len(df_BLFD_10C_1))
            
            #取电压数据一列
            df_BLFD_20C_V_1 = df_BLFD_20C.iloc[ : , 0] 
            np_BLFD_20C_V_1 = np.array(df_BLFD_20C_V_1)#np.ndarray()
            #将获取的电压数据转换为列表，方便后期写入excel
            BLFD_20C_V_1_list = np_BLFD_20C_V_1.tolist()#list
            #print(BLFD_40C_V_1_list)
            
            #取容量数据一列
            df_BLFD_20C_C_1 = df_BLFD_20C.iloc[ : , 1] 
            np_BLFD_20C_C_1 = np.array(df_BLFD_20C_C_1)#np.ndarray()
            #将获取的容量数据转换为列表，方便后期写入excel
            BLFD_20C_C_1_list = np_BLFD_20C_C_1.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            BLFD_20C_C_1 = BLFD_20C_C_1_list[-1]
            #数据列表长度
            BLFD_20C_C_1_list_len = len(BLFD_20C_C_1_list)
            #print(SWFD_C_1_list_len)
            #print(BLFD_40C_C_1_list)
            
            #取温度数据一列
            df_BLFD_20C_T_1 = df_BLFD_20C.iloc[ : , 2] 
            np_BLFD_20C_T_1 = np.array(df_BLFD_20C_T_1)#np.ndarray()
            #将获取的温度数据转换为列表，方便后期写入excel
            BLFD_20C_T_1_list = np_BLFD_20C_T_1.tolist()#list
            #取第一个和最后一个温度值作为计算当前温度下放电温升的数据
            BLFD_20C_T_1 = BLFD_20C_T_1_list[0]
            BLFD_20C_T_2 = BLFD_20C_T_1_list[-1]
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            BLFD_20C_T_2_list = []
            for i_BLFD_20C_T in range(0,BLFD_20C_C_1_list_len):
                BLFD_20C_T_2_list_i = BLFD_20C_T_1_list[i_BLFD_20C_T] - BLFD_20C_T_1
                BLFD_20C_T_2_list.append(BLFD_20C_T_2_list_i)
            #print(SWFD_T_2_list)
            #计算当前温度放电温升
            BLFD_20C_T_rise = round(BLFD_20C_T_2 - BLFD_20C_T_1, 2)
            #print(BLFD_40C_T_2_list)
            #print(BLFD_40C_T_1_list)
            
            
            #获取10C放电数据,和数据长度
            df_BLFD_10C_1_1 = df_BLFD_1[df_BLFD_1["电流(A)"] <= -50]
            df_BLFD_10C_1 = df_BLFD_10C_1_1[df_BLFD_10C_1_1["电流(A)"] > -150]
            len(df_BLFD_10C_1)
            #print(len(df_BLFD_10C_1))
            #print(df_BLFD_10C_1)
            #获取10C第一个数据序号
            BLFD_No_10C_1 = df_BLFD_10C_1.iloc[0, 2] - 1
            #获取10C待写入的数据
            df_BLFD_10C = df_BLFD_1.iloc[BLFD_No_10C_1 : BLFD_No_10C_1 + len(df_BLFD_10C_1), [4, 6, 7]]
            #截取新的数据，得到工步序号，方便截取1C数据
            df_BLFD_10C_1 = df_BLFD_1.iloc[BLFD_No_10C_1 : BLFD_No_10C_1 + len(df_BLFD_10C_1), [0, 4, 6, 7]]
            #print(df_BLFD_10C_1)
            
            #1C倍率工步序号
            df_BLFD_No_1C = df_BLFD_10C_1.iloc[0, 0] - 4
            #print(df_BLFD_No_1C)
            #print(BLFD_No_40C_1)
            #print(BLFD_No_40C_1 + len(df_BLFD_10C_1))
            
            #取电压数据一列
            df_BLFD_10C_V_1 = df_BLFD_10C.iloc[ : , 0] 
            np_BLFD_10C_V_1 = np.array(df_BLFD_10C_V_1)#np.ndarray()
            #将获取的电压数据转换为列表，方便后期写入excel
            BLFD_10C_V_1_list = np_BLFD_10C_V_1.tolist()#list
            #print(BLFD_40C_V_1_list)
            
            #取容量数据一列
            df_BLFD_10C_C_1 = df_BLFD_10C.iloc[ : , 1] 
            np_BLFD_10C_C_1 = np.array(df_BLFD_10C_C_1)#np.ndarray()
            #将获取的容量数据转换为列表，方便后期写入excel
            BLFD_10C_C_1_list = np_BLFD_10C_C_1.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            BLFD_10C_C_1 = BLFD_10C_C_1_list[-1]
            #数据列表长度
            BLFD_10C_C_1_list_len = len(BLFD_10C_C_1_list)
            #print(SWFD_C_1_list_len)
            #print(BLFD_40C_C_1_list)
            
            #取温度数据一列
            df_BLFD_10C_T_1 = df_BLFD_10C.iloc[ : , 2] 
            np_BLFD_10C_T_1 = np.array(df_BLFD_10C_T_1)#np.ndarray()
            #将获取的温度数据转换为列表，方便后期写入excel
            BLFD_10C_T_1_list = np_BLFD_10C_T_1.tolist()#list
            #取第一个和最后一个温度值作为计算当前温度下放电温升的数据
            BLFD_10C_T_1 = BLFD_10C_T_1_list[0]
            BLFD_10C_T_2 = BLFD_10C_T_1_list[-1]
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            BLFD_10C_T_2_list = []
            for i_BLFD_10C_T in range(0,BLFD_10C_C_1_list_len):
                BLFD_10C_T_2_list_i = BLFD_10C_T_1_list[i_BLFD_10C_T] - BLFD_10C_T_1
                BLFD_10C_T_2_list.append(BLFD_10C_T_2_list_i)
            #print(SWFD_T_2_list)
            #计算当前温度放电温升
            BLFD_10C_T_rise = round(BLFD_10C_T_2 - BLFD_10C_T_1, 2)
            #print(BLFD_40C_T_2_list)
            #print(BLFD_40C_T_1_list)
            
            
            
            #获取1C放电数据,和数据长度,通过10C的工步序号进行截取
            #数据工步序号：df_BLFD_No_1C
            
            df_BLFD_1C_1_1 = df_BLFD_1[df_BLFD_1["工步序号"] == df_BLFD_No_1C]
            #df_BLFD_1C_1 = df_BLFD_10C_1_1[df_BLFD_10C_1_1["电流(A)"] > -150]
            len(df_BLFD_1C_1_1)
            #print(len(df_BLFD_10C_1))
            #print(df_BLFD_1C_1_1)
            #获取1C第一个数据序号
            BLFD_No_1C_1 = df_BLFD_1C_1_1.iloc[0, 2] - 1
            #获取1C待写入的数据
            df_BLFD_1C = df_BLFD_1.iloc[BLFD_No_1C_1 : BLFD_No_1C_1 + len(df_BLFD_1C_1_1), [4, 6, 7]]
            #print(df_BLFD_1C)
            #print(BLFD_No_1C_1)
            #print(BLFD_No_40C_1 + len(df_BLFD_10C_1))
            
            #取电压数据一列
            df_BLFD_1C_V_1 = df_BLFD_1C.iloc[ : , 0] 
            np_BLFD_1C_V_1 = np.array(df_BLFD_1C_V_1)#np.ndarray()
            #将获取的电压数据转换为列表，方便后期写入excel
            BLFD_1C_V_1_list = np_BLFD_1C_V_1.tolist()#list
            #print(BLFD_40C_V_1_list)
            
            #取容量数据一列
            df_BLFD_1C_C_1 = df_BLFD_1C.iloc[ : , 1] 
            np_BLFD_1C_C_1 = np.array(df_BLFD_10C_C_1)#np.ndarray()
            #将获取的容量数据转换为列表，方便后期写入excel
            BLFD_1C_C_1_list = np_BLFD_1C_C_1.tolist()#list
            #取最后一个容量值作为当前温度下的放电容量，计算放电容量比率
            BLFD_1C_C_1 = BLFD_1C_C_1_list[-1]
            #数据列表长度
            BLFD_1C_C_1_list_len = len(BLFD_1C_C_1_list)
            #print(SWFD_C_1_list_len)
            #print(BLFD_40C_C_1_list)
            
            #取温度数据一列
            df_BLFD_1C_T_1 = df_BLFD_1C.iloc[ : , 2] 
            np_BLFD_1C_T_1 = np.array(df_BLFD_1C_T_1)#np.ndarray()
            #将获取的温度数据转换为列表，方便后期写入excel
            BLFD_1C_T_1_list = np_BLFD_1C_T_1.tolist()#list
            #取第一个和最后一个温度值作为计算当前温度下放电温升的数据
            BLFD_1C_T_1 = BLFD_1C_T_1_list[0]
            BLFD_1C_T_2 = BLFD_1C_T_1_list[-1]
            #生成新列表，以便计算放电温升
            #SWFD_T_0_list = [SWFD_T_1] * SWFD_C_1_list_len
            BLFD_1C_T_2_list = []
            for i_BLFD_1C_T in range(0,BLFD_1C_C_1_list_len):
                BLFD_1C_T_2_list_i = BLFD_1C_T_1_list[i_BLFD_1C_T] - BLFD_1C_T_1
                BLFD_1C_T_2_list.append(BLFD_1C_T_2_list_i)
            #print(BLFD_1C_T_2_list)
            #计算当前温度放电温升
            BLFD_1C_T_rise = round(BLFD_1C_T_2 - BLFD_1C_T_1, 2)
            #print(BLFD_1C_T_2_list)
            #print(BLFD_1C_T_1_list)
            
            
            
            
            
            
            
            #准备打开报告，写入倍率放电容量数据
            wb_BLFD = openpyxl.load_workbook("%s" % test_file_name_1)
            ws_BLFD = wb_BLFD["倍率放电"]
            
            #指定写入倍率放电的初始列索引
            k_BLFD_1 = counter_BLFD + 7 + (counter_BLFD - 1) * 19  #8\24
            
            #写入数据
            ws_BLFD.cell(row = 1,column = k_BLFD_1, value = BLFD_cell_num)
            for i_BLFD_5 in range(1, 5):
                #for col in range("%s" % row_DCR, "%s" % row_DCR)
                BLFD_col_list1 = ["1C容量（Ah）", "1C电压（V）", "1C温度（℃）", "1C温升（℃）"]
                BLFD_col_list2 = ["10C容量（Ah）", "10C电压（V）", "10C温度（℃）", "10C温升（℃）"]
                BLFD_col_list3 = ["20C容量（Ah）", "20C电压（V）", "20C温度（℃）", "20C温升（℃）"]
                BLFD_col_list4 = ["30C容量（Ah）", "30C电压（V）", "30C温度（℃）", "30C温升（℃）"]
                BLFD_col_list5 = ["40C容量（Ah）", "40C电压（V）", "40C温度（℃）", "40C温升（℃）"]
                ws_BLFD.cell(row = 2, column = k_BLFD_1+i_BLFD_5 - 1, value = BLFD_col_list1[i_BLFD_5 -1])
                ws_BLFD.cell(row = 2, column = k_BLFD_1+i_BLFD_5 + 3, value = BLFD_col_list2[i_BLFD_5 -1])
                ws_BLFD.cell(row = 2, column = k_BLFD_1+i_BLFD_5 + 7, value = BLFD_col_list3[i_BLFD_5 -1])
                ws_BLFD.cell(row = 2, column = k_BLFD_1+i_BLFD_5 + 11, value = BLFD_col_list4[i_BLFD_5 -1])
                ws_BLFD.cell(row = 2, column = k_BLFD_1+i_BLFD_5 + 15, value = BLFD_col_list5[i_BLFD_5 -1])
                
            for j_BLFD_1C in range(1, BLFD_1C_C_1_list_len+1):
                #写入1C容量、电压、温度、温升四列数据
                ws_BLFD.cell(row = j_BLFD_1C+2, column = k_BLFD_1, value = BLFD_1C_C_1_list[j_BLFD_1C-1]) 
                ws_BLFD.cell(row = j_BLFD_1C+2, column = k_BLFD_1+1, value = BLFD_1C_V_1_list[j_BLFD_1C-1])
                ws_BLFD.cell(row = j_BLFD_1C+2, column = k_BLFD_1+2, value = BLFD_1C_T_1_list[j_BLFD_1C-1])
                ws_BLFD.cell(row = j_BLFD_1C+2, column = k_BLFD_1+3, value = BLFD_1C_T_2_list[j_BLFD_1C-1])
                
            for j_BLFD_10C in range(1, BLFD_10C_C_1_list_len+1):
                #写入10C容量、电压、温度、温升四列数据
                ws_BLFD.cell(row = j_BLFD_10C+2, column = k_BLFD_1+4, value = BLFD_10C_C_1_list[j_BLFD_10C-1]) 
                ws_BLFD.cell(row = j_BLFD_10C+2, column = k_BLFD_1+5, value = BLFD_10C_V_1_list[j_BLFD_10C-1])
                ws_BLFD.cell(row = j_BLFD_10C+2, column = k_BLFD_1+6, value = BLFD_10C_T_1_list[j_BLFD_10C-1])
                ws_BLFD.cell(row = j_BLFD_10C+2, column = k_BLFD_1+7, value = BLFD_10C_T_2_list[j_BLFD_10C-1])
            for j_BLFD_20C in range(1, BLFD_20C_C_1_list_len+1):
                #写入20C容量、电压、温度、温升四列数据
                ws_BLFD.cell(row = j_BLFD_20C+2, column = k_BLFD_1+8, value = BLFD_20C_C_1_list[j_BLFD_20C-1]) 
                ws_BLFD.cell(row = j_BLFD_20C+2, column = k_BLFD_1+9, value = BLFD_20C_V_1_list[j_BLFD_20C-1])
                ws_BLFD.cell(row = j_BLFD_20C+2, column = k_BLFD_1+10, value = BLFD_20C_T_1_list[j_BLFD_20C-1])
                ws_BLFD.cell(row = j_BLFD_20C+2, column = k_BLFD_1+11, value = BLFD_20C_T_2_list[j_BLFD_20C-1])
                #写入30C容量、电压、温度、温升四列数据
            for j_BLFD_30C in range(1, BLFD_30C_C_1_list_len+1):
                ws_BLFD.cell(row = j_BLFD_30C+2, column = k_BLFD_1+12, value = BLFD_30C_C_1_list[j_BLFD_30C-1]) 
                ws_BLFD.cell(row = j_BLFD_30C+2, column = k_BLFD_1+13, value = BLFD_30C_V_1_list[j_BLFD_30C-1])
                ws_BLFD.cell(row = j_BLFD_30C+2, column = k_BLFD_1+14, value = BLFD_30C_T_1_list[j_BLFD_30C-1])
                ws_BLFD.cell(row = j_BLFD_30C+2, column = k_BLFD_1+15, value = BLFD_30C_T_2_list[j_BLFD_30C-1])
            for j_BLFD_40C in range(1, BLFD_40C_C_1_list_len+1):
                #写入40C容量、电压、温度、温升四列数据
                ws_BLFD.cell(row = j_BLFD_40C+2, column = k_BLFD_1+16, value = BLFD_40C_C_1_list[j_BLFD_40C-1]) 
                ws_BLFD.cell(row = j_BLFD_40C+2, column = k_BLFD_1+17, value = BLFD_40C_V_1_list[j_BLFD_40C-1])
                ws_BLFD.cell(row = j_BLFD_40C+2, column = k_BLFD_1+18, value = BLFD_40C_T_1_list[j_BLFD_40C-1])
                ws_BLFD.cell(row = j_BLFD_40C+2, column = k_BLFD_1+19, value = BLFD_40C_T_2_list[j_BLFD_40C-1])
                
            #设置放电容量表格格式
            for i_BLFD in ws_BLFD["A%s:G%s" % (counter_BLFD +2, counter_BLFD +2)]:
                for cell_BLFD in i_BLFD:
                    cell_BLFD.border = border  #区域单元格设置边框
                    cell_BLFD.alignment = Alignment(horizontal='center', vertical='center')
            #设置放电比率表格格式        
            for i_BLFD_1 in ws_BLFD["A%s:G%s" % (counter_BLFD +15, counter_BLFD +15)]:
                for cell_BLFD_1 in i_BLFD_1:
                    cell_BLFD_1.border = border  #区域单元格设置边框
                    cell_BLFD_1.alignment = Alignment(horizontal='center', vertical='center')
            #设置放电温升表格格式        
            for i_BLFD_2 in ws_BLFD["A%s:G%s" % (counter_BLFD +27, counter_BLFD +27)]:
                for cell_BLFD_2 in i_BLFD_2:
                    cell_BLFD_2.border = border  #区域单元格设置边框
                    cell_BLFD_2.alignment = Alignment(horizontal='center', vertical='center')    
            
            #写入容量表    
            ws_BLFD.cell(row = counter_BLFD +2,column = 1, value = counter_BLFD)
            ws_BLFD.cell(row = counter_BLFD +2,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_BLFD.cell(row = counter_BLFD +2,column = 2, value = BLFD_cell_num)
            ws_BLFD.cell(row = counter_BLFD +2,column = 3, value = BLFD_C)#初始容量
            ws_BLFD.cell(row = counter_BLFD +2,column = 4, value = BLFD_10C_C_1)#10C容量
            ws_BLFD.cell(row = counter_BLFD +2,column = 5, value = BLFD_20C_C_1)#20C容量
            ws_BLFD.cell(row = counter_BLFD +2,column = 6, value = BLFD_30C_C_1)#30C容量
            ws_BLFD.cell(row = counter_BLFD +2,column = 7, value = BLFD_40C_C_1)#40C容量
            
            
            
            
            
            #写入比率表    
            ws_BLFD.cell(row = counter_BLFD +15,column = 1, value = counter_BLFD)
            ws_BLFD.cell(row = counter_BLFD +15,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_BLFD.cell(row = counter_BLFD +15,column = 2, value = BLFD_cell_num)
            ws_BLFD.cell(row = counter_BLFD +15,column = 3, value = 1)#初始容量栏
            ws_BLFD.cell(row = counter_BLFD +15,column = 4, value = "=D%s/C%s" % (counter_BLFD +2, counter_BLFD +2))#10C比率
            ws_BLFD.cell(row = counter_BLFD +15,column = 5, value = "=E%s/C%s" % (counter_BLFD +2, counter_BLFD +2))#20C比率
            ws_BLFD.cell(row = counter_BLFD +15,column = 6, value = "=F%s/C%s" % (counter_BLFD +2, counter_BLFD +2))#30C比率
            ws_BLFD.cell(row = counter_BLFD +15,column = 7, value = "=G%s/C%s" % (counter_BLFD +2, counter_BLFD +2))#40C比率
            
            
            
            
            #写入温升表   
            ws_BLFD.cell(row = counter_BLFD +27,column = 1, value = counter_BLFD)
            ws_BLFD.cell(row = counter_BLFD +27,column = 2).alignment = Alignment(horizontal='right', vertical='center')
            ws_BLFD.cell(row = counter_BLFD +27,column = 2, value = BLFD_cell_num)
            ws_BLFD.cell(row = counter_BLFD +27,column = 3, value = BLFD_1C_T_rise)
            ws_BLFD.cell(row = counter_BLFD +27,column = 4, value = BLFD_10C_T_rise)
            ws_BLFD.cell(row = counter_BLFD +27,column = 5, value = BLFD_20C_T_rise)
            ws_BLFD.cell(row = counter_BLFD +27,column = 6, value = BLFD_30C_T_rise)
            ws_BLFD.cell(row = counter_BLFD +27,column = 7, value = BLFD_40C_T_rise)
            
            #计数器+1
            counter_BLFD += 1

            #print(ws_SWFD)
            wb_BLFD.save("%s" % test_file_name_1)
            print("1#" * 30)
            print("1#" * 30)
                   

        else:
            pass
        num += 1
        print(num)
    
    
    
        
            
          



#print(file_path)
#print(file_name_list1)


1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
C:\Users\Administrator\Desktop\jupyter notebook
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
读取第 0 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\DCR\01DCB01F010021A54D000030.xls
DCR
准备处理DCR数据
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
1
读取第 1 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\DCR\01DCB01F010021A54D000042.xls
DCR
准备处理DCR数据
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
2
读取第 2 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\DCR\01DCB01F010021A54D000079.xls
DCR
准备处理DCR数据
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
3
读取第 3 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\DCR\01DCB01F010021A54D000102.xls
DCR
准备处理DCR数据
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
4
读取第 4 个文件..........
C:\Users\Administr

1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
27
读取第 27 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\不同温度放电\-30℃\01DCB01F010021A54N000662-20200608-PXG-10AH-T-30T-DWFD-3_Channel_4_Wb_1.xlsx
不同温度放电\-30℃
准备调用不同温度放电-30℃.py
01DCB01F010021A54N000662
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
28
读取第 28 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\不同温度放电\-30℃\01DCB01F010021A54N000690-20200608-PXG-10AH-T-30T-DWFD-3_Channel_5_Wb_1.xlsx
不同温度放电\-30℃
准备调用不同温度放电-30℃.py
01DCB01F010021A54N000690
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
29
读取第 29 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\不同温度放电\-30℃\01DCB01F010021A54N000710-20200608-PXG-10AH-T-30T-DWFD-2_Chan

1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
51
读取第 51 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\不同温度放电\25℃\01DCB01F010021A54N000662-彭雪刚10Ah  不同温度放电  初始容量+ 满充.xls
不同温度放电\25℃
准备处理不同温度放电25℃数据
01DCB01F010021A54N000662
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
52
读取第 52 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\不同温度放电\25℃\01DCB01F010021A54N000690-彭雪刚10Ah  不同温度放电  初始容量+ 满充.xls
不同温度放电\25℃
准备处理不同温度放电25℃数据
01DCB01F010021A54N000690
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#1#
53
读取第 53 个文件..........
C:\Users\Administrator\Desktop\2020060301-VFFA50161227S18(3.2V10Ah)\不同温度放电\25℃\01DCB01F010021A54N000710-彭雪刚10Ah  不同温度放电  初始容量+ 满充.xls
不同温度放电\25℃
准备处理不同温度放电25℃数据
01DCB01F010021A54N000710
1#1#